# Determine anchor role
We've detected anchor use in posts, but can we determine what the role of the information is for the author/audience?

In [1]:
import numpy as np
import pandas as pd

## Load data

In [2]:
# data_file = '../../data/mined_tweets/combined_tweet_tag_data_NE_flat.gz'
# combined_data = pd.read_csv(data_file, index_col=False, sep='\t', compression='gzip')
# combined_data = combined_data.assign(**{'username' : combined_data.loc[:, 'username'].apply(lambda x: x.split(':')[-1])})
# combined_data.to_csv(data_file, sep='\t', index=False, compression='gzip')

In [108]:
data_file = '../../data/mined_tweets/combined_tweet_tag_data_NE_flat.gz'
combined_data = pd.read_csv(data_file, index_col=False, sep='\t', compression='gzip')
# remove bad NEs
data_name_NEs = ['#%s'%(x) for x in combined_data.loc[:, 'data_name_fixed'].unique()]
combined_data = combined_data[~combined_data.loc[:, 'NE_fixed'].isin(data_name_NEs)]
# restrict to valid
combined_data_valid = combined_data[combined_data.loc[:, 'valid_loc']==1]
display(combined_data_valid.head())

,id,txt,data_name_fixed,username,date,lang,NE,NE_type,NE_LOC,valid_loc,NE_fixed,has_descriptor,NE_fixed_clean,max_population,max_alternate_name_count,max_population_anchor,max_population_diff,max_alternate_name_count_anchor,max_alternate_name_count_diff
17,899098735367647232,Tropical Depression #Harvey is 1543 miles SSE ...,harvey,wralweather,2017-08-19 22:40:00,en,Raleigh,LOCATION,True,True,raleigh,False,raleigh,451066.0,43.0,False,0.0,False,0.0
31,899258634223353856,"#Harvey , #Illinois #firefighters ' #pension o...",harvey,acklaw,2017-08-20 09:15:00,en,Chicago,LOCATION,True,True,chicago,False,chicago,2720546.0,70.0,False,0.0,False,0.0
33,899272853954101249,"Harvey's. 2380 Wyandotte Street West, Windsor,...",harvey,foodpages,2017-08-20 10:12:00,en,Windsor,LOCATION,True,True,windsor,False,windsor,28778.0,16.0,False,0.0,False,0.0
49,899425632215588864,NHC_Atlantic: #Harvey 's remnants are likely t...,harvey,JasonBoyerWLOS,2017-08-20 20:19:00,en,Bay,LOCATION,True,True,bay,False,bay,15402.0,15.0,False,0.0,False,0.0
55,899443698693529601,@NHC_Atlantic #Harvey 's remnants are likely t...,harvey,BinTaxi,2017-08-20 21:31:00,en,Bay,LOCATION,True,True,bay,False,bay,15402.0,15.0,False,0.0,False,0.0


In [106]:
combined_data_valid.loc[:, 'username'].iloc[0]

'wralweather'

## Annotate anchors
Annotate true/false positives for anchoring.

In [572]:
import numpy as np
import os
np.random.seed(123)
anchor_col = 'max_population_anchor'
combined_data_valid_anchor = combined_data_valid[combined_data_valid.loc[:, anchor_col]==1]
group_var = 'data_name_fixed'
id_col = 'id'
N_sample = 50
NE_col = 'NE_fixed'
sample_NE_txt = []
for data_name_i, data_i in combined_data_valid.groupby(group_var):
    print('testing data=%s'%(data_name_i))
    data_i_anchor = data_i[data_i.loc[:, id_col].isin(combined_data_valid_anchor.loc[:, id_col].values)]
    data_i_anchor_ids = np.random.choice(data_i_anchor.loc[:, id_col].values, N_sample, replace=False)
    data_i_anchor_groups = data_i_anchor[data_i_anchor.loc[:, id_col].isin(data_i_anchor_ids)]
    print('%d/%d'%(data_i_anchor.shape[0], data_i.shape[0]))
    for data_j_id, data_j in data_i_anchor_groups.groupby(id_col):
        NE_to_anchor = data_j[data_j.loc[:, anchor_col]==1].loc[:, NE_col].values[0]
        print("NE to anchor = %s"%(NE_to_anchor))
        print("txt = %s"%(data_j.loc[:, 'txt'].iloc[0]))
        ## write to file because this is getting annoying to annotate
        sample_NE_txt.append([data_j_id, NE_to_anchor, data_j.loc[:, 'txt'].iloc[0], data_name_i])
sample_NE_txt = pd.DataFrame(sample_NE_txt, columns=['id', 'NE', 'txt', 'data_name'])
out_file_name = '../../data/mined_tweets/combined_tweet_tag_data_NE_flat_anchor_examples.tsv'
if(not os.path.exists(out_file_name)):
    sample_NE_txt.to_csv(out_file_name, sep='\t', index=False, float_format='%.3f')

testing data=florence
4477/8683
NE to anchor = new_bern
txt = RT @EdValleeWx: Our models specifically used for forecasting hurricanes have great agreement in #Florence making landfall near New Bern, NC…
NE to anchor = wilmington
txt = RT @WMO: Hurricane #Florence is likely to make landfall near Wilmington (North Carolina) . The tidal data shows a sea level rise of around…
NE to anchor = cape_fear
txt = RT @ABC: LATEST: Hurricane #Florence a Category 4 storm 670 miles ESE of Cape Fear, North Carolina, with maximum sustained winds of 140 mph…
NE to anchor = fayetteville
txt = You put your lips on them cigars more than you do on me" GF says. #cigar #lastselfie #sotl #botl #hurricaneflorence @ Fayetteville, North Carolina <URL>
NE to anchor = wilmington
txt = RT @FoxNewsResearch: #HurricaneFlorence - latest •Cat 2 •170 mi ESE of Wilmington, NC •220 mi E of Myrtle Beach, SC •10M+ in its path •1.7M…
NE to anchor = red_cross
txt = Met Reyna and her whole family who evacuated from the coast to

18355/70963
NE to anchor = corpus_christi
txt = The first Hurricane Watches for this part of #Texas in 3,267 days (9yrs!) like Houston & Corpus Christi #TXwx #Harvey h/t @KathrynProciv <URL>
NE to anchor = rockport
txt = As of 10 a.m., the modeling has #Harvey making landfall north of Rockport, which would spare Corpus Christi the worst of the storm.
NE to anchor = brownsville
txt = The eye of #HurricaneHarvey is now showing on the Brownsville, Texas long range radar. <URL>
NE to anchor = corpus_christi
txt = Praying for Texas, esp my other home Corpus Christi. #HurricaneHarvey
NE to anchor = matagorda
txt = Heavy squall headed toward Matagorda and Brazoria County coasts. Watch for waterspouts. #txwx #harvey <URL>
NE to anchor = galveston
txt = #Live . @mikebettes on #Periscope : Streaming cam from Galveston, Texas as Hurricane #Harvey approaches coast. #txwx <URL>
NE to anchor = galveston
txt = Headed to Galveston as #Harvey pushes closer to the TX coast. Look for updates here & live 

NE to anchor = miami
txt = @AmericanAir supposed to fly to the Dominican Republic, connection in Miami. Trying to switch to Mexico & you want to charge me? #Help #Irma
NE to anchor = tampa
txt = Hoping for a safe week for our friends at Hillsborough Area Regional Transit in Tampa, FL as they prepare for #HurricaneIrma .
NE to anchor = anna_maria_island
txt = Anna Maria Island evacuating 9/7/17 at 12:40 PM. Mileage check zero. Destination Clearwater then sweet home Alabama. #Irma please .
NE to anchor = miami
txt = My thoughts and prayers are with Miami and the entire State of Florida #HurricaneIrma
NE to anchor = white_house
txt = #HurricaneIrma please go hit Washington DC *Specifically White House* Thanks the hole world
NE to anchor = jacksonville
txt = JUST IN - A MANDATORY EVACUATION has been issued for Jacksonville, Florida. #CNN #Irma
NE to anchor = fort_myers
txt = Made it to Tulsa last night from Fort Myers escaping #hurricaneirma . Just home we have a home back in FL when we ge

NE to anchor = guaynabo
txt = More destruction photos from Bayamón, Guaynabo and San Juan #Maria <URL>
NE to anchor = humacao
txt = Si usted se dializa con Fresenius. Caguas, Rio Grande y Humacao van a estar ofreciendo servicios desde las 10am #puertorico
NE to anchor = altagracia
txt = #HuracanMaria Se deja sentir en Samana, Nagua, Santiago, La Altagracia, Santo Domingo Este y otras localidades. #Maria <URL>
NE to anchor = san_juan
txt = @weatherchannel is there any way we can hear of other towns in Puerto Rico? There aren’t only people in San Juan! #PuertoRico
NE to anchor = yabucoa
txt = RT UnivisionNews: Exclusive video: The devastating path of Hurricane #Maria across #Puerto Rico, from Yabucoa to San Juan
NE to anchor = jayuya
txt = Seeking info on my aunt Milly Bodon & Luis in Jayuya, Puerto Rico (Cuabey). Its only accessible by helicopter.No power/phone #HurricaneMaria <URL>
NE to anchor = san_juan
txt = #SanJuan #PuertoRico Drone footage shows flooded streets in San Juan, Puert

NE to anchor = panama_city
txt = #HappyMonday - If you think your Monday's bad... be reminded by the pics from #HurricaneMichael that it can ALWAYS get worse. Listen, we are here in Panama City, FL at groundZero delivering supplies to those impacted... <URL>
NE to anchor = panama_city
txt = The people ravaged by #HurricaneMichael in Panama City, Florida and Mexico Beach, Florida need our help. Let’s do this by donating to the American Red Cross.
NE to anchor = youngstown
txt = Day #2 of Hurricane #Michael Damage Surveys - Unbelievable tree damage. We surveyed spots in Southport, Youngstown, & Resota Beach (north of Lynn Haven & Panama City) where all trees were snapped, uprooted, or bent/twisted. Pictures do NOT do the damage justice at all... <URL>
NE to anchor = callaway
txt = #HurricaneMichael | Tide Loads of Hope is washing clothes for free from 9am-5pm at Walmart Supercenter (25 N Tyndall Pkwy, Callaway, FL 32404) #PanhandleStrong #PanamaCity <URL>
NE to anchor = calhoun
txt = Che

In [569]:
sample_NE_txt.loc[:, 'id'].dtype

dtype('int64')

## After annotation

In [44]:
# after annotating
sample_NE_txt = pd.read_csv('../../data/mined_tweets/combined_tweet_tag_data_NE_flat_anchor_examples.tsv', sep='\t', index_col=False)
# regular context
print('%d/%d'%(sample_NE_txt.loc[:, 'context'].sum(), sample_NE_txt.shape[0]))
print(sample_NE_txt.groupby('data_name').apply(lambda x: x.loc[:, 'context'].sum()))
# sub-clause context
print('%d/%d'%(sample_NE_txt.loc[:, 'context_subclause'].sum(), sample_NE_txt.shape[0]))
print(sample_NE_txt.groupby('data_name').apply(lambda x: x.loc[:, 'context_subclause'].sum()))

103/248
data_name
florence    29
harvey      17
irma        12
maria       19
michael     26
dtype: int64
20/248
data_name
florence    6
harvey      7
irma        0
maria       3
michael     4
dtype: int64


This distribution is not great! Less than 50% of anchors are actually used for disambiguation, and less than 10% of anchors exist as sub-clauses (i.e. can be extracted by parses). Maybe we need a better definition of "anchor," like [occur within K words of the less important NE].

### Recover subclause context from parse
Can we recover subclause context using dependency parses?

We can try spacy to start. The heuristic will be "does the smaller NE have a subclause that contains the anchor NE"?

In [223]:
from importlib import reload
import parse_twitter_data
reload(parse_twitter_data)
from parse_twitter_data import parse_data
## parse all the data! because we need to keep the "invalid" locations
## that are used for anchoring, ex. state names
combined_data_parsed = parse_data(combined_data.drop_duplicates('id', inplace=False))
# np.random.seed(123)
# test_size = 100
# test_data = combined_data_valid.loc[np.random.choice(combined_data_valid.index, test_size, replace=False), :]
# test_parsed = parse_data(test_data)

/hg190/istewart6/miniconda3/envs/crisis_language/lib/python3.6/site-packages/pandas/core/indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s


In [224]:
## reorganize: one line per ID
combined_data_parsed_flat = []
for id_i, data_i in combined_data_parsed.groupby('id'):
    data_i_flat = pd.Series([id_i, list(data_i.loc[:, 'parse'])])
    combined_data_parsed_flat.append(data_i_flat)
combined_data_parsed_flat = pd.concat(combined_data_parsed_flat, axis=1).transpose()
combined_data_parsed_flat.columns = ['id', 'parse']
## combine
combined_data_parsed_flat = pd.merge(combined_data, combined_data_parsed_flat, on='id')

In [225]:
print('%d/%d data'%(combined_data.shape[0], combined_data_parsed_flat.shape[0]))

498786/498786 data


In [226]:
# get some test data
# need >1 NEs, at least one anchor
anchor_var = 'max_population_anchor'
valid_var = 'valid_loc'
data_anchor_ids = combined_data_parsed_flat[combined_data_parsed_flat.loc[:, anchor_var]==1].loc[:, 'id'].unique()
# at least one status in group should be valid ;_;
anchor_data = combined_data_parsed_flat[combined_data_parsed_flat.loc[:, 'id'].isin(data_anchor_ids)]
anchor_data = pd.concat([data_i for id_i, data_i in anchor_data.groupby('id') if data_i.loc[:, valid_var].sum(axis=0) > 0.], axis=0)
anchor_data.sort_values(['data_name_fixed', 'id'], inplace=True)
display(anchor_data.iloc[:10, :])

,id,txt,data_name_fixed,username,date,lang,NE,NE_type,NE_LOC,valid_loc,NE_fixed,has_descriptor,NE_fixed_clean,max_population,max_alternate_name_count,max_population_anchor,max_population_diff,max_alternate_name_count_anchor,max_alternate_name_count_diff,parse
453202,1038453206693695489,RT @BMcNoldy: Only 10 major hurricanes have ma...,florence,thebutterknife,2018-09-08 15:45:12+00:00,en,Jacksonville,LOCATION,True,True,jacksonville,False,jacksonville,868031.0,56.0,False,0.0,False,0.0,"[[[RT, PROPN, 1, compound, 0], [@BMcNoldy, PRO..."
453203,1038453206693695489,RT @BMcNoldy: Only 10 major hurricanes have ma...,florence,thebutterknife,2018-09-08 15:45:12+00:00,en,Cape_Hatteras,LOCATION,True,False,cape_hatteras,False,cape hatteras,0.0,0.0,True,868031.0,True,56.0,"[[[RT, PROPN, 1, compound, 0], [@BMcNoldy, PRO..."
453262,1038802428655620096,RT @MelissaNordWx: New 11 AM Advisory from the...,florence,OldCrowNest,2018-09-09 14:52:53+00:00,en,Wilmington,LOCATION,True,True,wilmington,True,wilmington,115933.0,38.0,True,8495434.0,True,129.0,"[[[RT, PROPN, 1, compound, 0], [@MelissaNordWx..."
453263,1038802428655620096,RT @MelissaNordWx: New 11 AM Advisory from the...,florence,OldCrowNest,2018-09-09 14:52:53+00:00,en,NC,LOCATION,True,False,nc,False,nc,8611367.0,167.0,False,0.0,False,0.0,"[[[RT, PROPN, 1, compound, 0], [@MelissaNordWx..."
453292,1038943919298502659,Shoutout to #HurricaneFlorence for ruining my ...,florence,Candace_RDH,2018-09-10 00:15:07+00:00,en,Charleston,LOCATION,True,True,charleston,True,charleston,132609.0,53.0,True,10456416.0,True,68.0,"[[[Shoutout, PROPN, 9, nsubj, 0], [to, PART, 0..."
453293,1038943919298502659,Shoutout to #HurricaneFlorence for ruining my ...,florence,Candace_RDH,2018-09-10 00:15:07+00:00,en,TN,LOCATION,True,False,tn,False,tn,10589025.0,121.0,False,0.0,False,0.0,"[[[Shoutout, PROPN, 9, nsubj, 0], [to, PART, 0..."
453337,1039158004946530305,RT @NWSCharlestonSC: Hurricane #Florence was c...,florence,ChasersChs,2018-09-10 14:25:49+00:00,en,Charleston,LOCATION,True,True,charleston,True,charleston,132609.0,53.0,True,4097233.0,True,122.0,"[[[RT, PROPN, 1, compound, 0], [@NWSCharleston..."
453338,1039158004946530305,RT @NWSCharlestonSC: Hurricane #Florence was c...,florence,ChasersChs,2018-09-10 14:25:49+00:00,en,SC,LOCATION,True,False,sc,False,sc,4229842.0,175.0,False,0.0,False,0.0,"[[[RT, PROPN, 1, compound, 0], [@NWSCharleston..."
453374,1039207250282274816,RT @EdValleeWx: Population density map overlai...,florence,kathycalm,2018-09-10 17:41:30+00:00,en,Wilmington,LOCATION,True,True,wilmington,False,wilmington,115933.0,38.0,True,711164.0,True,13.0,"[[[RT, PROPN, 1, compound, 0], [@EdValleeWx, P..."
453375,1039207250282274816,RT @EdValleeWx: Population density map overlai...,florence,kathycalm,2018-09-10 17:41:30+00:00,en,Charlotte,LOCATION,True,True,charlotte,False,charlotte,827097.0,51.0,False,0.0,False,0.0,"[[[RT, PROPN, 1, compound, 0], [@EdValleeWx, P..."


We want to capture: 
- NE + appositive subclause ("Little River in Manchester, NC", "Harris County in Texas")
- NE + parent noun + appositive subclause ("Mayfair neighborhood of Lumberton, NC")
- NE + child conjunction + appositive subclause ("for Quebradillas and Isabela, PR")

In [340]:
## extract subclause anchor
import re
import data_helpers
reload(data_helpers)
from data_helpers import extract_NE_subtree, extract_full_tree
def detect_subclause_anchor(data, valid_var='valid_loc', 
                            anchor_var='max_population', NE_var='NE', 
                            allowed_child_dep_types=['acl', 'appos', 'prep', 'nummod'],
                            allowed_parent_dep_types=['nmod', 'compound', 'pobj'],
                            verbose=False):
    # find children for each NE
    data_NEs = data.loc[:, NE_var].values
    # fix format
    data_NEs = [x.replace('_', ' ') for x in data_NEs]
    data_anchor_vals = data.loc[:, anchor_var].values
    parent_subclause_anchor = []
    subclause_anchor = []
    trees = data.loc[:, 'parse'].iloc[0]
    tree_graphs = [extract_full_tree(t) for t in trees]
    tree_ctr = 0
    tree_token_ctr = 0
    for i, data_NE in enumerate(data_NEs):
        if(verbose):
            print('candidate %d=%s'%(i, data_NE))
        anchor_val_i = data_anchor_vals[i]
        anchor_NE_candidates = [x for x,y in zip(data_NEs, data_anchor_vals) if y > anchor_val_i]
        if(verbose):
            print('anchor NE candidates = %s'%(','.join(anchor_NE_candidates)))
        parent_subclause_anchor_i = 0
        subclause_anchor_i = 0
        if(len(anchor_NE_candidates) > 0):
            
            # add whitespace to candidates to avoid matching substrings
            anchor_NE_candidate_matcher = re.compile('|'.join([' %s'%(x) for x in anchor_NE_candidates]))
            data_NE_tree, data_NE_subtree, tree_ctr, tree_token_ctr = extract_NE_subtree(data_NE, trees, tree_graphs, tree_ctr, tree_token_ctr)
            # in case tree counter is incremented too far
            tree_ctr = min(tree_ctr, len(trees)-1)

            ## test 1: determine if sibling contains anchor NE
            if(len(data_NE_tree) > 0):
                if(verbose):
                    print('data NE tree=%s'%(str(data_NE_tree)))
                # find parent node for noun, preposition tests
                # parent node => index that is not included in NE indices
#                 highest_child_node = sorted(data_NE_tree, key=lambda x: x[2])[-1]
                data_NE_tree_idx = [x[4] for x in data_NE_tree]
                NE_child_nodes = [x for x in data_NE_tree if x[2] not in data_NE_tree_idx]
                if(len(NE_child_nodes) > 0):
                    highest_child_node = NE_child_nodes[0]
                    parent_node_idx = highest_child_node[2]
                    parent_node_dep = highest_child_node[3]
                    parent_node = trees[tree_ctr][parent_node_idx]
                    if(verbose):
                        print('NE parse token at tree=%d, token=%d:'%(tree_ctr, tree_token_ctr))
                        print(str(trees[tree_ctr][tree_token_ctr-1]))
                        print('NE parent token:')
                        print(str(parent_node))
                    # if parent is noun, find children and look for anchor NEs
                    if(parent_node_dep in allowed_parent_dep_types):
        #             if(parent_node[1]=='NOUN'):
                        parent_node_subtree_idx = get_subtree(parent_node[4], tree_graphs[tree_ctr])
                        parent_node_subtree = [x for x in trees[tree_ctr] if x[4] in parent_node_subtree_idx]
                        if(verbose):
                            print('parent node subtree %s'%(str(parent_node_subtree)))
                        # check for anchor NE
                        parent_node_subtree_str = ' '.join([x[0] for x in parent_node_subtree])
                        parent_subclause_anchor_i = int(anchor_NE_candidate_matcher.search(parent_node_subtree_str) is not None)
                    # special case: if parent node string matches candidate
                    # then we count it as anchor e.g. "CITY, STATE" pattern
                    if(parent_node[0] in anchor_NE_candidates):
                        parent_subclause_anchor_i = 1
            ## test 2: determine if child contains anchor NE
            if(len(data_NE_subtree) > 0):
                if(verbose):
                    print('NE=%s subtree=%s'%(data_NE, str(data_NE_subtree)))
                ## filter for allowed dependency trees
                ## find dep type for highest node (lowest parent index) in subtree
                min_node_dep_idx = min(data_NE_subtree, key=lambda x: x[2])
                min_node_deps = [x[3] for x in data_NE_subtree if x[2]==min_node_dep_idx[2]]
                if(verbose):
                    print('min node deps %s'%(str(min_node_deps)))
                if(len(set(min_node_deps) & set(allowed_child_dep_types)) > 0):
                    ## look for NE in phrase
                    subtree_str = ' '.join([x[0] for x in data_NE_subtree])
                    if(verbose):
                        print('subtree = %s'%(subtree_str))
                    subclause_anchor_i = int(anchor_NE_candidate_matcher.search(subtree_str) is not None)
                    ## conjuction is a special case: anchor NE must occur 
                    ## in format NE CONJ NE, NE_anchor
                    if('conj' in min_node_deps):
                        anchor_NE_candidate_matcher_conj = re.compile('|'.join([',\s?%s'%(x) for x in anchor_NE_candidates]))
        parent_subclause_anchor.append(parent_subclause_anchor_i)
        subclause_anchor.append(subclause_anchor_i)
    parent_subclause_anchor = pd.Series(parent_subclause_anchor, index=data_NEs)
    subclause_anchor = pd.Series(subclause_anchor, index=data_NEs)
    return parent_subclause_anchor, subclause_anchor

In [289]:
import data_helpers
reload(data_helpers)
from data_helpers import extract_NE_subtree, extract_full_tree, get_subtree
tree_ctr = 0
tree_token_ctr = 0

## test case 1: conjunction
# test_NE = 'Mexico beach'
# test_trees = [[['Want', 'VERB', 0, 'ROOT', 0], ['to', 'PART', 2, 'aux', 1], ['see', 'VERB', 0, 'xcomp', 2], ['the', 'DET', 5, 'det', 3], ['real', 'ADJ', 5, 'amod', 4], ['#AHSApocalypse', 'NOUN', 2, 'dobj', 5], ['?', 'PUNCT', 0, 'punct', 6]], [['Come', 'VERB', 0, 'ROOT', 0], ['down', 'PART', 0, 'prt', 1], ['here', 'ADV', 0, 'advmod', 2], ['to', 'ADP', 0, 'prep', 3], ['Mexico', 'PROPN', 5, 'compound', 4], ['beach', 'NOUN', 3, 'pobj', 5], ['or', 'CCONJ', 5, 'cc', 6], ['Panama', 'PROPN', 9, 'compound', 7], ['City', 'PROPN', 9, 'compound', 8], ['FL', 'PROPN', 5, 'conj', 9], ['!', 'PUNCT', 0, 'punct', 10]], [['#HurricaneMichael', 'PROPN', 2, 'compound', 0], ['#hurricanemichael2018', 'PUNCT', 2, 'nsubj', 1], ['@CNN', 'PROPN', 2, 'ROOT', 2], ['@weatherchannel', 'X', 2, 'punct', 3]]]
# print(str(test_trees))
# test_tree_graphs = [extract_full_tree(t) for t in test_trees]
# test_NE_tree, test_NE_subtree, tree_ctr, tree_token_ctr = extract_NE_subtree(test_NE, test_trees, test_tree_graphs, tree_ctr, tree_token_ctr, verbose=True)

## test case 2: parent noun
# test_NE = 'Mayfair'
# test_trees = [[['Homes', 'NOUN', 8, 'nsubj', 0], ['in', 'ADP', 0, 'prep', 1], ['the', 'DET', 4, 'det', 2], ['Mayfair', 'PROPN', 4, 'compound', 3], ['neighborhood', 'NOUN', 1, 'pobj', 4], ['of', 'ADP', 4, 'prep', 5], ['Lumberton', 'PROPN', 7, 'compound', 6], ['NC', 'PROPN', 5, 'pobj', 7], ['are', 'VERB', 8, 'ROOT', 8], ['underwater', 'ADJ', 8, 'acomp', 9], ['again', 'ADV', 8, 'advmod', 10], ['after', 'ADP', 8, 'prep', 11], ['being', 'VERB', 13, 'auxpass', 12], ['flooded', 'VERB', 11, 'pcomp', 13], ['in', 'ADP', 13, 'prep', 14], ['2016', 'NUM', 14, 'pobj', 15], ['during', 'ADP', 13, 'prep', 16], ['Hurricane', 'PROPN', 18, 'compound', 17], ['Matthew', 'PROPN', 16, 'pobj', 18], ['..', 'PUNCT', 8, 'punct', 19], ['and', 'CCONJ', 8, 'cc', 20], ['water', 'NOUN', 22, 'compound', 21], ['levels', 'NOUN', 23, 'nsubj', 22], ['continue', 'VERB', 8, 'conj', 23], ['to', 'PART', 25, 'aux', 24], ['rise', 'VERB', 23, 'xcomp', 25], ['.', 'PUNCT', 23, 'punct', 26]], [['#HurricaneFlorence', 'PROPN', 2, 'compound', 0], ['#lumbertonnc', 'PROPN', 2, 'compound', 1], ['#northcarolina', 'PROPN', 2, 'ROOT', 2], ['#lumberriver', 'PROPN', 2, 'appos', 3], ['@TheNOWtv', 'PROPN', 2, 'punct', 4]]]
# test_tree_graphs = [extract_full_tree(t) for t in test_trees]
# print(str(test_trees))
# test_NE_tree, test_NE_subtree, tree_ctr, tree_token_ctr = extract_NE_subtree(test_NE, test_trees, test_tree_graphs, tree_ctr, tree_token_ctr, verbose=True)
# parent_node_idx = min([x[2] for x in test_NE_tree])
# parent_node = test_trees[tree_ctr][parent_node_idx]
# print('parent node=%s'%(parent_node))
# parent_node_subtree_idx = get_subtree(parent_node[4], test_tree_graphs[tree_ctr])
# parent_node_subtree = [x for x in test_trees[tree_ctr] if x[4] in parent_node_subtree_idx]
# print('parent node subtree=%s'%(str(parent_node_subtree)))

## test case 3: parent preposition
test_NE = 'Little River'
test_trees = [[['The', 'DET', 2, 'det', 0], ['Little', 'PROPN', 2, 'compound', 1], ['River', 'PROPN', 14, 'nsubj', 2], ['in', 'ADP', 2, 'prep', 3], ['Manchester', 'PROPN', 5, 'compound', 4], ['NC', 'PROPN', 3, 'pobj', 5], ['near', 'ADP', 2, 'prep', 6], ['Spring', 'PROPN', 8, 'compound', 7], ['Lake', 'PROPN', 6, 'pobj', 8], ['and', 'CCONJ', 6, 'cc', 9], ['upstream', 'NOUN', 6, 'conj', 10], ['from', 'ADP', 10, 'prep', 11], ['#FayettevilleNC', 'PROPN', 11, 'pobj', 12], ['has', 'VERB', 14, 'aux', 13], ['hit', 'VERB', 14, 'ROOT', 14], ['a', 'DET', 17, 'det', 15], ['record', 'NOUN', 17, 'compound', 16], ['level', 'NOUN', 14, 'dobj', 17], ['according', 'VERB', 14, 'prep', 18], ['to', 'ADP', 18, 'prep', 19], ['@NWS', 'X', 19, 'pobj', 20], ['.', 'PUNCT', 14, 'punct', 21]], [['River', 'NOUN', 0, 'ROOT', 0], ['at', 'ADP', 0, 'prep', 1], ['34.96', 'NUM', 3, 'nummod', 2], ['ft', 'NOUN', 1, 'pobj', 3], ['as', 'ADP', 0, 'prep', 4], ['of', 'ADP', 4, 'prep', 5], ['3:30', 'NUM', 7, 'nummod', 6], ['PM', 'NOUN', 5, 'pobj', 7], ['Monday', 'PROPN', 0, 'npadvmod', 8], ['.', 'PUNCT', 0, 'punct', 9]], [['During', 'ADP', 9, 'prep', 0], ['Hurricane', 'PROPN', 2, 'compound', 1], ['Matthew', 'PROPN', 0, 'pobj', 2], ['(', 'PUNCT', 6, 'punct', 3], ['previous', 'ADJ', 6, 'amod', 4], ['alltime', 'NOUN', 6, 'amod', 5], ['high', 'ADJ', 2, 'appos', 6], [')', 'PUNCT', 6, 'punct', 7], ['it', 'PRON', 9, 'nsubj', 8], ['reached', 'VERB', 9, 'ROOT', 9], ['32.19', 'NUM', 11, 'nummod', 10], ['ft', 'NOUN', 9, 'dobj', 11], ['.', 'PUNCT', 9, 'punct', 12], ['#florence', 'X', 14, 'compound', 13], ['#ncwx', 'PROPN', 14, 'ROOT', 14], ['@newsobserver', 'X', 14, 'punct', 15]]]
test_tree_graphs = [extract_full_tree(t) for t in test_trees]
print(str(test_trees))
test_NE_tree, test_NE_subtree, tree_ctr, tree_token_ctr = extract_NE_subtree(test_NE, test_trees, test_tree_graphs, tree_ctr, tree_token_ctr, verbose=True)
parent_node_idx = min([x[2] for x in test_NE_tree])
parent_node = test_trees[tree_ctr][parent_node_idx]
print('parent node=%s'%(parent_node))
parent_node_subtree_idx = get_subtree(parent_node[4], test_tree_graphs[tree_ctr])
parent_node_subtree = [x for x in test_trees[tree_ctr] if x[4] in parent_node_subtree_idx]
print('parent node subtree=%s'%(str(parent_node_subtree)))

[[['The', 'DET', 2, 'det', 0], ['Little', 'PROPN', 2, 'compound', 1], ['River', 'PROPN', 14, 'nsubj', 2], ['in', 'ADP', 2, 'prep', 3], ['Manchester', 'PROPN', 5, 'compound', 4], ['NC', 'PROPN', 3, 'pobj', 5], ['near', 'ADP', 2, 'prep', 6], ['Spring', 'PROPN', 8, 'compound', 7], ['Lake', 'PROPN', 6, 'pobj', 8], ['and', 'CCONJ', 6, 'cc', 9], ['upstream', 'NOUN', 6, 'conj', 10], ['from', 'ADP', 10, 'prep', 11], ['#FayettevilleNC', 'PROPN', 11, 'pobj', 12], ['has', 'VERB', 14, 'aux', 13], ['hit', 'VERB', 14, 'ROOT', 14], ['a', 'DET', 17, 'det', 15], ['record', 'NOUN', 17, 'compound', 16], ['level', 'NOUN', 14, 'dobj', 17], ['according', 'VERB', 14, 'prep', 18], ['to', 'ADP', 18, 'prep', 19], ['@NWS', 'X', 19, 'pobj', 20], ['.', 'PUNCT', 14, 'punct', 21]], [['River', 'NOUN', 0, 'ROOT', 0], ['at', 'ADP', 0, 'prep', 1], ['34.96', 'NUM', 3, 'nummod', 2], ['ft', 'NOUN', 1, 'pobj', 3], ['as', 'ADP', 0, 'prep', 4], ['of', 'ADP', 4, 'prep', 5], ['3:30', 'NUM', 7, 'nummod', 6], ['PM', 'NOUN', 5, 'p

### Test anchor extraction on annotated data

In [354]:
from time import sleep
# get test IDs from annotated data
annotated_anchor_data = pd.read_csv('../../data/mined_tweets/combined_tweet_tag_data_NE_flat_anchor_examples.tsv', sep='\t', index_col=False)
annotated_anchor_NEs = annotated_anchor_data[annotated_anchor_data.loc[:, 'context_subclause']==1].loc[:, 'NE']
annotated_anchor_txt = annotated_anchor_data[annotated_anchor_data.loc[:, 'context_subclause']==1].loc[:, 'txt']
## hard code 
subclause_anchor = []
subclause_parent_anchor = []
subclause_combined_anchor = []
for annotated_anchor_NE_i, txt_i in zip(annotated_anchor_NEs, annotated_anchor_txt.values):
    data_i = anchor_data[anchor_data.loc[:, 'txt']==txt_i]
    id_i = data_i.loc[:, 'id'].iloc[0]
    NE_i = data_i.loc[:, 'NE_fixed'].values
#     print('id=%d; NE=%s'%(id_i, ','.join(NE_i)))
#     print('txt=%s'%(txt_i))
    ## goal: get parse data, extract the subclause, determine if anchoring NE exists
    parse_i = data_i.loc[:, 'parse'].iloc[0]
#     print('parse=%s'%(str(parse_i)))
    parent_subclause_anchor_i, subclause_anchor_i = detect_subclause_anchor(data_i, verbose=False)
    parent_subclause_anchor_i.index = [x.replace(' ','_').lower() for x in parent_subclause_anchor_i.index]
    subclause_anchor_i.index = [x.replace(' ','_').lower() for x in subclause_anchor_i.index]
    anchor_final_i = pd.concat([parent_subclause_anchor_i, subclause_anchor_i], axis=1).max(axis=1)
#     print('parent subclause anchors = \n%s'%(str(parent_subclause_anchor_i)))
#     print('subclause anchors = \n%s'%(str(subclause_anchor_i)))
    # combine parent/subclause results
#     anchor_final_i = pd.concat([parent_subclause_anchor_i, subclause_anchor_i], axis=1).max(axis=1)
    # fix index to match annotated format
#     anchor_final_i.index = [x.replace(' ','_').lower() for x in anchor_final_i.index]
    subclause_anchor.append(subclause_anchor_i.loc[annotated_anchor_NE_i])
    subclause_parent_anchor.append(parent_subclause_anchor_i.loc[annotated_anchor_NE_i])
    subclause_combined_anchor.append(anchor_final_i.loc[annotated_anchor_NE_i])

print('%d/%d anchors detected with subclause results; recall=%.3f'%(sum(subclause_anchor), len(subclause_combined_anchor), sum(subclause_anchor) / len(subclause_combined_anchor)))
print('%d/%d anchors detected with parent subclause results; recall=%.3f'%(sum(subclause_parent_anchor), len(subclause_combined_anchor), sum(subclause_parent_anchor) / len(subclause_combined_anchor)))
print('%d/%d anchors detected with combined results; recall=%.3f'%(sum(subclause_combined_anchor), len(subclause_combined_anchor), sum(subclause_combined_anchor) / len(subclause_combined_anchor)))

## compute precision over all annotated data
subclause_anchor_estimate = []
parent_subclause_anchor_estimate = []
subclause_combined_anchor_estimate = []
subclause_anchor_gold = []
for idx_i, annotated_data_i in annotated_anchor_data.iterrows():
    annotated_NE_i = annotated_data_i.loc['NE']
#     print('NE=%s'%(annotated_NE_i))
    txt_i = annotated_data_i.loc['txt']
    data_i = anchor_data[anchor_data.loc[:, 'txt']==txt_i]
    data_NE_i = data_i.loc[:, 'NE_fixed'].values
    # restrict to data where NE only occurs once per status
    if(len([x for x in data_NE_i if x.replace(' ', '_').lower()==annotated_NE_i]) == 1):
        parent_subclause_anchor_i, subclause_anchor_i = detect_subclause_anchor(data_i, verbose=False)
        parent_subclause_anchor_i.index = [x.replace(' ','_').lower() for x in parent_subclause_anchor_i.index]
        subclause_anchor_i.index = [x.replace(' ','_').lower() for x in subclause_anchor_i.index]
        anchor_final_i = pd.concat([parent_subclause_anchor_i, subclause_anchor_i], axis=1).max(axis=1)
        parent_subclause_anchor_estimate.append(parent_subclause_anchor_i.loc[annotated_NE_i])
        subclause_anchor_estimate.append(subclause_anchor_i.loc[annotated_NE_i])
        subclause_combined_anchor_estimate.append(anchor_final_i.loc[annotated_NE_i])
        subclause_anchor_gold.append(annotated_data_i.loc['context_subclause'])
subclause_anchor_estimate = np.array(subclause_anchor_estimate)
parent_subclause_anchor_estimate = np.array(parent_subclause_anchor_estimate)
subclause_combined_anchor_estimate = np.array(subclause_combined_anchor_estimate)
subclause_anchor_gold = np.array(subclause_anchor_gold)
# TP = annotated_anchor_data.loc[:, 'context_subclause'].values && subclause_anchor_full
sub_prec = 1 - abs(subclause_anchor_gold - subclause_anchor_estimate).sum() / len(subclause_anchor_gold)
parent_prec = 1 - abs(subclause_anchor_gold - parent_subclause_anchor_estimate).sum() / len(subclause_anchor_gold)
prec = 1 - abs(subclause_anchor_gold - subclause_combined_anchor_estimate).sum() / len(subclause_anchor_gold)
print('subclause precision=%.3f'%(sub_prec))
print('parent subclause precision=%.3f'%(parent_prec))
print('combined precision=%.3f'%(prec))
# anchor_data_i = anchor_data[anchor_data.loc[:, 'id']==id_i]
# display(anchor_data_i.head())
# display(anchor_data_i.loc[:, 'txt'].iloc[0])

6/20 anchors detected with subclause results; recall=0.300
15/20 anchors detected with parent subclause results; recall=0.750
17/20 anchors detected with combined results; recall=0.850
subclause precision=0.934
parent subclause precision=0.616
combined precision=0.616


OK! We see that including parent subclauses improves recall but hurts precision. No surprises there.

Let's look at the false positives that the parent strategy captures and try to weed those out.

In [359]:
for idx_i, annotated_data_i in annotated_anchor_data.iterrows():
    annotated_NE_i = annotated_data_i.loc['NE']
    txt_i = annotated_data_i.loc['txt']
    data_i = anchor_data[anchor_data.loc[:, 'txt']==txt_i]
    data_NE_i = data_i.loc[:, 'NE_fixed'].values
    # restrict to data where NE only occurs once per status
    if(len([x for x in data_NE_i if x.replace(' ', '_').lower()==annotated_NE_i]) == 1):
        print('txt=%s'%(txt_i))
        parent_subclause_anchor_i, subclause_anchor_i = detect_subclause_anchor(data_i, verbose=True)
        parent_subclause_anchor_i.index = [x.replace(' ','_').lower() for x in parent_subclause_anchor_i.index]
        if(parent_subclause_anchor_i.loc[annotated_NE_i] - annotated_data_i.loc['context_subclause'] == 1):
            print('false positive with NE=%s, txt=%s'%(annotated_NE_i, txt_i))

txt=RT @EdValleeWx: Our models specifically used for forecasting hurricanes have great agreement in #Florence making landfall near New Bern, NC…
candidate 0=New Bern
anchor NE candidates = NC
data NE tree=[['New', 'PROPN', 20, 'compound', 18], ['Bern', 'PROPN', 20, 'compound', 19]]
NE parse token at tree=0, token=20:
['Bern', 'PROPN', 20, 'compound', 19]
NE parent token:
['NC', 'PROPN', 17, 'pobj', 20]
parent node subtree [['New', 'PROPN', 20, 'compound', 18], ['Bern', 'PROPN', 20, 'compound', 19]]
candidate 1=NC
anchor NE candidates = 
false positive with NE=new_bern, txt=RT @EdValleeWx: Our models specifically used for forecasting hurricanes have great agreement in #Florence making landfall near New Bern, NC…
txt=RT @WMO: Hurricane #Florence is likely to make landfall near Wilmington (North Carolina) . The tidal data shows a sea level rise of around…
candidate 0=Wilmington
anchor NE candidates = North Carolina
data NE tree=[['Wilmington', 'PROPN', 10, 'pobj', 11]]
NE parse token at t

txt=Downtown New Bern, North Carolina #Florence #1010WINS #NewBernStrong <URL>
candidate 0=New Bern
anchor NE candidates = North Carolina
data NE tree=[['New', 'PROPN', 2, 'compound', 1], ['Bern', 'PROPN', 5, 'compound', 2]]
NE parse token at tree=0, token=3:
['Bern', 'PROPN', 5, 'compound', 2]
NE parent token:
['#Florence', 'ADP', 5, 'ROOT', 5]
parent node subtree [['Downtown', 'NOUN', 2, 'compound', 0], ['New', 'PROPN', 2, 'compound', 1], ['Bern', 'PROPN', 5, 'compound', 2], ['North', 'PROPN', 4, 'compound', 3], ['Carolina', 'PROPN', 5, 'compound', 4], ['#1010WINS', 'PROPN', 7, 'compound', 6], ['#NewBernStrong', 'X', 5, 'appos', 7]]
NE=New Bern subtree=[['Downtown', 'NOUN', 2, 'compound', 0]]
min node deps ['compound']
candidate 1=North Carolina
anchor NE candidates = 
false positive with NE=new_bern, txt=Downtown New Bern, North Carolina #Florence #1010WINS #NewBernStrong <URL>
txt=#HurricaneFlorence update! @ Charlotte, North Carolina <URL>
candidate 0=Charlotte
anchor NE candidate

txt=After almost 30"" of rain during #Florence , Jones county is now experiencing historic flooding along the Trent River. Here r some pics from Pollocksville where rescue crews have worked tirelessly assisting residents. Special thank you to @NYPDnews , who have come a long way 2 help! <URL>
candidate 0=Jones county
anchor NE candidates = 
candidate 1=Trent River
anchor NE candidates = Jones county,Pollocksville
data NE tree=[['Trent', 'PROPN', 19, 'compound', 18], ['River', 'PROPN', 16, 'pobj', 19]]
NE parse token at tree=0, token=20:
['River', 'PROPN', 16, 'pobj', 19]
NE parent token:
['along', 'ADP', 15, 'prep', 16]
parent node subtree [['the', 'DET', 19, 'det', 17], ['Trent', 'PROPN', 19, 'compound', 18], ['River', 'PROPN', 16, 'pobj', 19]]
NE=Trent River subtree=[['the', 'DET', 19, 'det', 17]]
min node deps ['det']
candidate 2=Pollocksville
anchor NE candidates = Jones county
data NE tree=[['Pollocksville', 'PROPN', 4, 'pobj', 5]]
NE parse token at tree=1, token=6:
['Pollocksvill

txt=. @COJacksonville NC​ officials recommend avoiding a major intersection at the end of the Jacksonville bypass because of heavy traffic use by those heading to the southern coast. #HurricaneFlorence #FlorenceNC <URL>
candidate 0=NC
anchor NE candidates = 
candidate 1=Jacksonville
anchor NE candidates = NC
data NE tree=[['Jacksonville', 'PROPN', 15, 'compound', 14]]
NE parse token at tree=1, token=15:
['Jacksonville', 'PROPN', 15, 'compound', 14]
NE parent token:
['bypass', 'NOUN', 12, 'pobj', 15]
parent node subtree [['the', 'DET', 15, 'det', 13], ['Jacksonville', 'PROPN', 15, 'compound', 14]]
candidate 2=southern coast
anchor NE candidates = NC,Jacksonville
data NE tree=[['southern', 'ADJ', 27, 'amod', 26], ['coast', 'NOUN', 24, 'pobj', 27]]
NE parse token at tree=1, token=28:
['coast', 'NOUN', 24, 'pobj', 27]
NE parent token:
['to', 'ADP', 23, 'prep', 24]
parent node subtree [['the', 'DET', 27, 'det', 25], ['southern', 'ADJ', 27, 'amod', 26], ['coast', 'NOUN', 24, 'pobj', 27]]
NE=

txt=#BREAKING #HurricaneHarvey makes landfall on TX coast over the northern end of San Jose Island about 4 miles east of Rockport. #nprnewscast
candidate 0=TX
anchor NE candidates = 
candidate 1=San Jose Island
anchor NE candidates = TX,Rockport
data NE tree=[['San', 'PROPN', 13, 'compound', 12], ['Jose', 'PROPN', 14, 'compound', 13], ['Island', 'PROPN', 11, 'pobj', 14]]
NE parse token at tree=0, token=15:
['Island', 'PROPN', 11, 'pobj', 14]
NE parent token:
['of', 'ADP', 10, 'prep', 11]
parent node subtree [['San', 'PROPN', 13, 'compound', 12], ['Jose', 'PROPN', 14, 'compound', 13], ['Island', 'PROPN', 11, 'pobj', 14]]
candidate 2=Rockport
anchor NE candidates = TX
data NE tree=[['Rockport', 'PROPN', 19, 'pobj', 20]]
NE parse token at tree=0, token=21:
['Rockport', 'PROPN', 19, 'pobj', 20]
NE parent token:
['of', 'ADP', 18, 'prep', 19]
parent node subtree [['Rockport', 'PROPN', 19, 'pobj', 20]]
txt=#HurricaneHarvey makes landfall on San Jose Island, TX near Rockport, TX as a Cat. 4 hu

...

txt=Puerto Rico humanitarian crisis Trump talking about Wall Street + banks San Juan Mayor Carmen Yulin Cruz #Maria <URL>
candidate 0=Puerto Rico
anchor NE candidates = 
candidate 1=San Juan
anchor NE candidates = Puerto Rico
data NE tree=[['San', 'PROPN', 12, 'compound', 11], ['Juan', 'PROPN', 13, 'compound', 12]]
NE parse token at tree=0, token=13:
['Juan', 'PROPN', 13, 'compound', 12]
NE parent token:
['Mayor', 'PROPN', 13, 'ROOT', 13]
parent node subtree [['San', 'PROPN', 12, 'compound', 11], ['Juan', 'PROPN', 13, 'compound', 12]]
txt=#PuertoRico Some towns in Puerto Rico have been able to set up hotlines to find family: San Juan 787-294-0277... <URL>
candidate 0=Puerto Rico
anchor NE candidates = 
candidate 1=San Juan
anchor NE candidates = Puerto Rico
data NE tree=[['San', 'PROPN', 18, 'compound', 17], ['Juan', 'PROPN', 18, 'ROOT', 18]]
NE=San Juan subtree=[['7872940277', 'NUM', 18, 'nummod', 19], ['...', 'PUNCT', 18, 'punct', 20]]
min node deps ['nummod', 'punct']
subtree = 7872

txt=Posting for a friend: “If anyone knows what conditions are like or has pics near Delaware Ave (right outside the Lynn Haven Country Club) or on Lisenby near 390, Lynn Haven, FL north of Panama City Beach, FL. Still can get ahold of my mom or grandparents.” #HurricaneMichael
candidate 0=Delaware
anchor NE candidates = FL
data NE tree=[['Delaware', 'PROPN', 18, 'compound', 17]]
NE parse token at tree=0, token=18:
['Delaware', 'PROPN', 18, 'compound', 17]
NE parent token:
['Ave', 'PROPN', 16, 'pobj', 18]
parent node subtree [['Delaware', 'PROPN', 18, 'compound', 17], ['(', 'PUNCT', 18, 'punct', 19]]
candidate 1=Lynn Haven
anchor NE candidates = Delaware,FL
data NE tree=[['Lynn', 'PROPN', 26, 'compound', 23], ['Haven', 'PROPN', 26, 'compound', 24]]
NE parse token at tree=0, token=25:
['Haven', 'PROPN', 26, 'compound', 24]
NE parent token:
['Club', 'PROPN', 21, 'pobj', 26]
parent node subtree [['the', 'DET', 26, 'det', 22], ['Lynn', 'PROPN', 26, 'compound', 23], ['Haven', 'PROPN', 26, '

false positive with NE=panama_city_beach, txt=Thank you, this Panama City Beach #hurricanemichael survivor appreciates what you are doing for us!! Your store in Santa Rosa Beach helped us today when we drove over from Panama City to purchase food/water/supplies for our friends and coworkers. The employees were so kind! <URL>
txt=Rob Golding drove to Springfield, Florida to be with his 89-year-old dad during #HurricaneMichael . His home was spared, but the area suffered damage, so they started taking in neighbors, going out to rescue friends, and organizing food for the community. <URL>
candidate 0=Springfield
anchor NE candidates = Florida
data NE tree=[['Springfield', 'PROPN', 5, 'compound', 4]]
NE parse token at tree=0, token=5:
['Springfield', 'PROPN', 5, 'compound', 4]
NE parent token:
['Florida', 'PROPN', 3, 'pobj', 5]
parent node subtree [['Springfield', 'PROPN', 5, 'compound', 4]]
candidate 1=Florida
anchor NE candidates = 
false positive with NE=springfield, txt=Rob Golding dro

txt=#HurricaneMichael | Tide Loads of Hope is washing clothes for free from 9am-5pm at Walmart Supercenter (25 N Tyndall Pkwy, Callaway, FL 32404) #PanhandleStrong #PanamaCity <URL>
candidate 0=Callaway
anchor NE candidates = FL
data NE tree=[['Callaway', 'PROPN', 22, 'compound', 21]]
NE parse token at tree=0, token=22:
['Callaway', 'PROPN', 22, 'compound', 21]
NE parent token:
['FL', 'PROPN', 15, 'appos', 22]
parent node subtree [['25', 'NUM', 22, 'nummod', 17], ['N', 'NOUN', 22, 'compound', 18], ['Tyndall', 'PROPN', 22, 'compound', 19], ['Pkwy', 'PROPN', 22, 'compound', 20], ['Callaway', 'PROPN', 22, 'compound', 21], ['32404', 'NUM', 22, 'nummod', 23]]
candidate 1=FL
anchor NE candidates = 
false positive with NE=callaway, txt=#HurricaneMichael | Tide Loads of Hope is washing clothes for free from 9am-5pm at Walmart Supercenter (25 N Tyndall Pkwy, Callaway, FL 32404) #PanhandleStrong #PanamaCity <URL>
txt=Check out Calhoun (98% out), Gulf (86%), Jackson (83%), Liberty (71%), and Bay 

txt=Hey guys... PLEASE come to Panama City, Fl. We need some laughs after #HurricaneMichael The beach didn't get hit bad at all. There are many venues. Club La Vila is a huge one. We'd love to see ya'll. Please consider it.
candidate 0=Panama City
anchor NE candidates = Fl
data NE tree=[['Panama', 'PROPN', 5, 'compound', 3], ['City', 'PROPN', 5, 'compound', 4]]
NE parse token at tree=1, token=5:
['City', 'PROPN', 5, 'compound', 4]
NE parent token:
['Fl', 'PROPN', 2, 'pobj', 5]
parent node subtree [['Panama', 'PROPN', 5, 'compound', 3], ['City', 'PROPN', 5, 'compound', 4]]
candidate 1=Fl
anchor NE candidates = 
false positive with NE=panama_city, txt=Hey guys... PLEASE come to Panama City, Fl. We need some laughs after #HurricaneMichael The beach didn't get hit bad at all. There are many venues. Club La Vila is a huge one. We'd love to see ya'll. Please consider it.
txt=This afternoon, state and federal partners held a call to coordinate housing solutions in areas impacted by #Hurricane

OK! Based on manual inspection it looks like we need to capture the following categories:

- (child) child_dep=noun-modifier/preposition/apposition + LOC ("Guaynabo - one of the wealthiest towns in Puerto Rico")
- (child) child=conj + STATE ("Port Aransas and Port O'Conner, TX")
- (parent) NE_dep=compound/nmod + STATE ("Mayfair neighborhood of Lumberton, NC")

Let's re-write the detection algorithm.

In [463]:
import re
STATES_SHORT_FULL_LOOKUP = {
    'FL' : 'Florida', 'NC' : 'North Carolina', 'SC' : 'South Carolina', 
    'VA' : 'Virginia', 'GA' : 'Georgia', 'PR' : 'Puerto Rico',
    'LA' : 'Louisiana', 'TX' : 'Texas',
}
DATA_NAME_STATES_SHORT_LOOKUP = {
    'florence' : ['FL', 'NC', 'SC', 'VA', 'GA'],
    'irma' : ['FL', 'GA', 'SC'],
    'harvey' : ['TX', 'LA'],
    'maria' : ['PR', 'NC'],
    'michael' : ['FL', 'GA', 'NC', 'SC']
}
DATA_NAME_STATES_LONG_LOOKUP = {k : [STATES_SHORT_FULL_LOOKUP[v] for v in vs] for k, vs in DATA_NAME_STATES_SHORT_LOOKUP.items()}
DATA_NAME_STATES_LOOKUP = {k : DATA_NAME_STATES_SHORT_LOOKUP[k]+DATA_NAME_STATES_LONG_LOOKUP[k] for k in DATA_NAME_STATES_SHORT_LOOKUP.keys()}
DATA_NAME_STATES_MATCHERS = {k : re.compile('|'.join([' %s |^%s | %s$|^%s$'%((v1.lower(),)*4) for v1 in v])) for k,v in DATA_NAME_STATES_LOOKUP.items()}
from data_helpers import extract_NE_subtree, extract_full_tree
def detect_subclause_anchor_parents_children(data, valid_var='valid_loc', 
                                             anchor_var='max_population', NE_var='NE', 
                                             data_name_var='data_name_fixed',
                                             child_dep_types=['acl', 'appos', 'prep', 'nummod'],
                                             parent_dep_types=['nmod', 'compound'],
                                             verbose=False):
    data_NEs = data.loc[:, NE_var].values
    # fix format to match parse
    data_NEs = [x.replace('_', ' ') for x in data_NEs]
    data_valid = data.loc[:, valid_var].values
    data_NEs_valid = [x for x,y in zip(data_NEs, data_valid) if y==1]
    data_anchor_vals = data.loc[:, anchor_var].values
    parent_subclause_anchor = []
    subclause_anchor = []
    trees = data.loc[:, 'parse'].iloc[0]
    tree_graphs = [extract_full_tree(t) for t in trees]
    tree_ctr = 0
    tree_token_ctr = 0
    data_name = data.loc[:, data_name_var].iloc[0]
    state_matcher = DATA_NAME_STATES_MATCHERS[data_name]
    for i, data_NE in enumerate(data_NEs_valid):
        if(verbose):
            print('candidate %d=%s'%(i, data_NE))
        anchor_val_i = data_anchor_vals[i]
        anchor_NE_candidates = [x for x,y in zip(data_NEs, data_anchor_vals) if y > anchor_val_i]
        if(verbose):
            print('anchor NE candidates = %s'%(','.join(anchor_NE_candidates)))
        parent_subclause_anchor_i = 0
        subclause_anchor_i = 0
        if(len(anchor_NE_candidates) > 0):
            
            # add whitespace to candidates to avoid matching substrings e.g. "FL" doesn't match "FLOW"
            anchor_NE_candidate_matcher = re.compile('|'.join([' %s |^%s | %s$'%(x,x,x) for x in anchor_NE_candidates]))
            data_NE_tree, data_NE_subtree, tree_ctr, tree_token_ctr = extract_NE_subtree(data_NE, trees, tree_graphs, tree_ctr, tree_token_ctr)
            
            ## parent test: test if sibling contains state
            if(len(data_NE_tree) > 0):
                if(verbose):
                    print('data NE tree=%s'%(str(data_NE_tree)))
                # find parent node for noun, preposition tests
                # parent node => index that is not included in NE indices
                data_NE_tree_idx = [x[4] for x in data_NE_tree]
                NE_child_nodes = [x for x in data_NE_tree if x[2] not in data_NE_tree_idx]
                if(len(NE_child_nodes) > 0):
                    highest_child_node = NE_child_nodes[0]
                    parent_node_idx = highest_child_node[2]
                    parent_node_dep = highest_child_node[3]
                    parent_node = trees[tree_ctr][parent_node_idx]
                    if(verbose):
                        print('NE parse token at tree=%d, token=%d:'%(tree_ctr, tree_token_ctr))
                        print(str(trees[tree_ctr][tree_token_ctr-1]))
                        print('NE parent token:')
                        print(str(parent_node))
                    # if parent is connected by valid dep, find children and look for state mention
                    if(parent_node_dep in parent_dep_types):
                        parent_node_subtree_idx = get_subtree(parent_node[4], tree_graphs[tree_ctr])
                        parent_node_subtree = [x for x in trees[tree_ctr] if x[4] in parent_node_subtree_idx]
                        if(verbose):
                            print('parent node subtree %s'%(str(parent_node_subtree)))
                        parent_node_subtree_str = ' '.join([x[0] for x in parent_node_subtree]).lower()
#                         if(verbose):
#                             print('matching states %s on tree %s'%(state_matcher.pattern, parent_node_subtree_str.lower()))
                        parent_subclause_anchor_i = int(state_matcher.search(parent_node_subtree_str) is not None)
#                         parent_subclause_anchor_i = int(anchor_NE_candidate_matcher.search(parent_node_subtree_str) is not None)
                    # special case: if parent node string matches candidate
                    # then we count it as anchor e.g. "CITY, STATE" pattern
#                     if(parent_node[0] in anchor_NE_candidates):
#                         parent_subclause_anchor_i = 1
            
            ## child test: test if child contains anchor
            if(len(data_NE_subtree) > 0):
                if(verbose):
                    print('NE=%s subtree=%s'%(data_NE, str(data_NE_subtree)))
                ## filter for allowed dependency trees
                ## find dep type for highest node (lowest parent index) in subtree
                min_node_dep_idx = min(data_NE_subtree, key=lambda x: x[2])
                min_node_deps = [x[3] for x in data_NE_subtree if x[2]==min_node_dep_idx[2]]
                if(verbose):
                    print('min node deps %s'%(str(min_node_deps)))
                subtree_str = ' '.join([x[0] for x in data_NE_subtree])
                if(len(set(min_node_deps) & set(child_dep_types)) > 0):
                    ## look for NE in phrase
                    if(verbose):
                        print('subtree = %s'%(subtree_str))
                    subclause_anchor_i = int(anchor_NE_candidate_matcher.search(subtree_str) is not None)
                    ## conjuction is a special case: anchor NE must occur 
                    ## in format NE CONJ NE, state
                elif('conj' in min_node_deps):
                    subclause_anchor_i = int(state_matcher.search(subtree_str.lower()) is not None)
        parent_subclause_anchor.append(parent_subclause_anchor_i)
        subclause_anchor.append(subclause_anchor_i)
    parent_subclause_anchor = pd.Series(parent_subclause_anchor, index=data_NEs_valid)
    subclause_anchor = pd.Series(subclause_anchor, index=data_NEs_valid)
    return parent_subclause_anchor, subclause_anchor

In [384]:
## load annotated
annotated_anchor_data = pd.read_csv('../../data/mined_tweets/combined_tweet_tag_data_NE_flat_anchor_examples.tsv', sep='\t', index_col=False)
annotated_anchor_NEs = annotated_anchor_data[annotated_anchor_data.loc[:, 'context_subclause']==1].loc[:, 'NE']
annotated_anchor_txt = annotated_anchor_data[annotated_anchor_data.loc[:, 'context_subclause']==1].loc[:, 'txt']
## track all types of anchors
subclause_anchor = []
subclause_parent_anchor = []
subclause_combined_anchor = []
for annotated_anchor_NE_i, txt_i in zip(annotated_anchor_NEs, annotated_anchor_txt.values):
    data_i = anchor_data[anchor_data.loc[:, 'txt']==txt_i]
    id_i = data_i.loc[:, 'id'].iloc[0]
    NE_i = data_i.loc[:, 'NE_fixed'].values
#     print('id=%d; NE=%s'%(id_i, ','.join(NE_i)))
#     print('txt=%s'%(txt_i))
    ## goal: get parse data, extract the subclause, determine if anchoring NE exists
    parse_i = data_i.loc[:, 'parse'].iloc[0]
#     print('parse=%s'%(str(parse_i)))
    parent_subclause_anchor_i, subclause_anchor_i = detect_subclause_anchor_parents_children(data_i, verbose=False)
    parent_subclause_anchor_i.index = [x.replace(' ','_').lower() for x in parent_subclause_anchor_i.index]
    subclause_anchor_i.index = [x.replace(' ','_').lower() for x in subclause_anchor_i.index]
    anchor_final_i = pd.concat([parent_subclause_anchor_i, subclause_anchor_i], axis=1).max(axis=1)
#     print('NE=%s, subclause_anchor=%d, parent_subclause_anchor=%d'%
#           (annotated_anchor_NE_i, 
#            subclause_anchor_i.loc[annotated_anchor_NE_i],
#            parent_subclause_anchor_i.loc[annotated_anchor_NE_i]))
#     print('parent subclause anchors = \n%s'%(str(parent_subclause_anchor_i)))
#     print('subclause anchors = \n%s'%(str(subclause_anchor_i)))
    # combine parent/subclause results
#     anchor_final_i = pd.concat([parent_subclause_anchor_i, subclause_anchor_i], axis=1).max(axis=1)
    # fix index to match annotated format
#     anchor_final_i.index = [x.replace(' ','_').lower() for x in anchor_final_i.index]
    subclause_anchor.append(subclause_anchor_i.loc[annotated_anchor_NE_i])
    subclause_parent_anchor.append(parent_subclause_anchor_i.loc[annotated_anchor_NE_i])
    subclause_combined_anchor.append(anchor_final_i.loc[annotated_anchor_NE_i])

print('%d/%d anchors detected with subclause results; recall=%.3f'%(sum(subclause_anchor), len(subclause_combined_anchor), sum(subclause_anchor) / len(subclause_combined_anchor)))
print('%d/%d anchors detected with parent subclause results; recall=%.3f'%(sum(subclause_parent_anchor), len(subclause_combined_anchor), sum(subclause_parent_anchor) / len(subclause_combined_anchor)))
print('%d/%d anchors detected with combined results; recall=%.3f'%(sum(subclause_combined_anchor), len(subclause_combined_anchor), sum(subclause_combined_anchor) / len(subclause_combined_anchor)))

## compute precision over all annotated data
subclause_anchor_estimate = []
parent_subclause_anchor_estimate = []
subclause_combined_anchor_estimate = []
subclause_anchor_gold = []
for idx_i, annotated_data_i in annotated_anchor_data.iterrows():
    annotated_NE_i = annotated_data_i.loc['NE']
#     print('NE=%s'%(annotated_NE_i))
    txt_i = annotated_data_i.loc['txt']
    data_i = anchor_data[anchor_data.loc[:, 'txt']==txt_i]
    data_NE_i = data_i.loc[:, 'NE_fixed'].values
    # restrict to data where NE only occurs once per status
    if(len([x for x in data_NE_i if x.replace(' ', '_').lower()==annotated_NE_i]) == 1):
        parent_subclause_anchor_i, subclause_anchor_i = detect_subclause_anchor_parents_children(data_i, verbose=False)
        parent_subclause_anchor_i.index = [x.replace(' ','_').lower() for x in parent_subclause_anchor_i.index]
        subclause_anchor_i.index = [x.replace(' ','_').lower() for x in subclause_anchor_i.index]
        anchor_final_i = pd.concat([parent_subclause_anchor_i, subclause_anchor_i], axis=1).max(axis=1)
        parent_subclause_anchor_estimate.append(parent_subclause_anchor_i.loc[annotated_NE_i])
        subclause_anchor_estimate.append(subclause_anchor_i.loc[annotated_NE_i])
        subclause_combined_anchor_estimate.append(anchor_final_i.loc[annotated_NE_i])
        subclause_anchor_gold.append(annotated_data_i.loc['context_subclause'])
subclause_anchor_estimate = np.array(subclause_anchor_estimate)
parent_subclause_anchor_estimate = np.array(parent_subclause_anchor_estimate)
subclause_combined_anchor_estimate = np.array(subclause_combined_anchor_estimate)
subclause_anchor_gold = np.array(subclause_anchor_gold)
# TP = annotated_anchor_data.loc[:, 'context_subclause'].values && subclause_anchor_full
sub_prec = 1 - abs(subclause_anchor_gold - subclause_anchor_estimate).sum() / len(subclause_anchor_gold)
parent_prec = 1 - abs(subclause_anchor_gold - parent_subclause_anchor_estimate).sum() / len(subclause_anchor_gold)
prec = 1 - abs(subclause_anchor_gold - subclause_combined_anchor_estimate).sum() / len(subclause_anchor_gold)
print('subclause precision=%.3f'%(sub_prec))
print('parent subclause precision=%.3f'%(parent_prec))
print('combined precision=%.3f'%(prec))
# anchor_data_i = anchor_data[anchor_data.loc[:, 'id']==id_i]
# display(anchor_data_i.head())
# display(anchor_data_i.loc[:, 'txt'].iloc[0])

10/20 anchors detected with subclause results; recall=0.500
5/20 anchors detected with parent subclause results; recall=0.250
15/20 anchors detected with combined results; recall=0.750
subclause precision=0.942
parent subclause precision=0.905
combined precision=0.930


In [387]:
for idx_i, annotated_data_i in annotated_anchor_data.iterrows():
    annotated_NE_i = annotated_data_i.loc['NE']
    txt_i = annotated_data_i.loc['txt']
    data_i = anchor_data[anchor_data.loc[:, 'txt']==txt_i]
    data_NE_i = data_i.loc[:, 'NE_fixed'].values
    # restrict to data where NE only occurs once per status
    if(len([x for x in data_NE_i if x.replace(' ', '_').lower()==annotated_NE_i]) == 1):
        print('txt=%s'%(txt_i))
        parent_subclause_anchor_i, subclause_anchor_i = detect_subclause_anchor_parents_children(data_i, verbose=True)
        subclause_anchor_i.index = [x.replace(' ', '_').lower() for x in subclause_anchor_i.index]
        parent_subclause_anchor_i.index = [x.replace(' ','_').lower() for x in parent_subclause_anchor_i.index]
        anchor_final_i = pd.concat([parent_subclause_anchor_i, subclause_anchor_i], axis=1).max(axis=1)
        if(anchor_final_i.loc[annotated_NE_i] - annotated_data_i.loc['context_subclause'] == -1):
            print('false negative with NE=%s, txt=%s'%(annotated_NE_i, txt_i))

txt=RT @EdValleeWx: Our models specifically used for forecasting hurricanes have great agreement in #Florence making landfall near New Bern, NC…
candidate 0=New Bern
anchor NE candidates = NC
data NE tree=[['New', 'PROPN', 20, 'compound', 18], ['Bern', 'PROPN', 20, 'compound', 19]]
NE parse token at tree=0, token=20:
['Bern', 'PROPN', 20, 'compound', 19]
NE parent token:
['NC', 'PROPN', 17, 'pobj', 20]
parent node subtree [['New', 'PROPN', 20, 'compound', 18], ['Bern', 'PROPN', 20, 'compound', 19]]
txt=RT @WMO: Hurricane #Florence is likely to make landfall near Wilmington (North Carolina) . The tidal data shows a sea level rise of around…
candidate 0=Wilmington
anchor NE candidates = North Carolina
data NE tree=[['Wilmington', 'PROPN', 10, 'pobj', 11]]
NE parse token at tree=0, token=12:
['Wilmington', 'PROPN', 10, 'pobj', 11]
NE parent token:
['near', 'ADP', 9, 'prep', 10]
NE=Wilmington subtree=[['(', 'PUNCT', 11, 'punct', 12], ['North', 'PROPN', 14, 'compound', 13], ['Carolina', 'PR

txt=Luckily both boxes were delivered and were delayed by #hurricaneflorence but I fly out tomorrow for a weeks vacation in Canada (Montreal and Quebec) then to #nyc for a week for work and… <URL>
candidate 0=Montreal
anchor NE candidates = Quebec
data NE tree=[['Montreal', 'PROPN', 20, 'appos', 22]]
NE parse token at tree=0, token=23:
['Montreal', 'PROPN', 20, 'appos', 22]
NE parent token:
['Canada', 'PROPN', 19, 'pobj', 20]
NE=Montreal subtree=[['and', 'CCONJ', 22, 'cc', 23], ['Quebec', 'PROPN', 22, 'conj', 24]]
min node deps ['cc', 'conj']
candidate 1=Quebec
anchor NE candidates = 
txt=The remains of #Florence spared Rochester. However, it's a different story from Ithaca to Oneonta. 3 to 4"" of rain has fallen from Chemung to southern Cortland counties. Flood Warnings are up for some there. <URL>
candidate 0=Ithaca
anchor NE candidates = 
txt=Yesterday, our crews teamed up with @insideFPL to assess damage throughout Columbus and Bladen counties in North Carolina. Crews continue to w

txt=Coal ash flowing like pudding in Neuse River near Duke's Goldsboro power plant <URL>
candidate 0=Duke
anchor NE candidates = Duke,Goldsboro
candidate 1=Goldsboro
anchor NE candidates = Goldsboro
data NE tree=[['Goldsboro', 'PROPN', 12, 'compound', 10]]
NE parse token at tree=0, token=11:
['Goldsboro', 'PROPN', 12, 'compound', 10]
NE parent token:
['plant', 'NOUN', 8, 'pobj', 12]
parent node subtree [["Duke's", 'PROPN', 12, 'compound', 9], ['Goldsboro', 'PROPN', 12, 'compound', 10], ['power', 'NOUN', 12, 'compound', 11]]
txt=The first Hurricane Watches for this part of #Texas in 3,267 days (9yrs!) like Houston & Corpus Christi #TXwx #Harvey h/t @KathrynProciv <URL>
candidate 0=Houston
anchor NE candidates = 
candidate 1=Corpus Christi
anchor NE candidates = Houston
data NE tree=[['Corpus', 'PROPN', 5, 'compound', 3], ['Christi', 'PROPN', 5, 'compound', 4]]
NE parse token at tree=1, token=5:
['Christi', 'PROPN', 5, 'compound', 4]
NE parent token:
['#TXwx', 'PROPN', 1, 'conj', 5]
pare

txt=Wolff said patients from Ben Taub in Houston might be transported to San Antonio area hospitals. Not confirmed yet. #Harvey
candidate 0=Houston
anchor NE candidates = Houston,San Antonio
data NE tree=[['Houston', 'PROPN', 6, 'pobj', 7]]
NE parse token at tree=0, token=8:
['Houston', 'PROPN', 6, 'pobj', 7]
NE parent token:
['in', 'ADP', 5, 'prep', 6]
candidate 1=San Antonio
anchor NE candidates = 
txt=Plano efforts w/ #HurricaneHarvey (1/2) - Plano Fire-Rescue: 2 members deployed w/ TX Task Force One & 3 (incl K-9) w/ TX Task Force Two. <URL>
candidate 0=Plano
anchor NE candidates = TX,TX
data NE tree=[['Plano', 'PROPN', 1, 'compound', 0]]
NE parse token at tree=0, token=1:
['Plano', 'PROPN', 1, 'compound', 0]
NE parent token:
['efforts', 'NOUN', 1, 'ROOT', 1]
parent node subtree [['Plano', 'PROPN', 1, 'compound', 0]]
txt=VIDEO: I-10 at Yale, The Heights, Houston (residential area on the north side of downtown.) ( - @euzkera ) #Harvey <URL>
candidate 0=Yale
anchor NE candidates = Ho

txt=We're gonna be headed to Woodsboro, TX pop. 1,512 tomorrow to drop off supplies. #Harvey Amazon Wish List - <URL>
candidate 0=Woodsboro
anchor NE candidates = TX
data NE tree=[['Woodsboro', 'PROPN', 6, 'compound', 5]]
NE parse token at tree=0, token=6:
['Woodsboro', 'PROPN', 6, 'compound', 5]
NE parent token:
['TX', 'PROPN', 7, 'compound', 6]
parent node subtree [['Woodsboro', 'PROPN', 6, 'compound', 5]]
txt=@kellycass Good morning from Morgantown,WV. Watching today's rain & that impact on Cheat River watershed before #Irma arrives.
candidate 0=Morgantown
anchor NE candidates = WV
txt=@ShiriSpear do you think #IRMA will directly hit North Florida / Orlando / WDW area. Flying from London to Orlando Thursday curious to kno?
candidate 0=Orlando
anchor NE candidates = Orlando,London,Orlando
data NE tree=[['Orlando', 'PROPN', 13, 'nmod', 11]]
NE parse token at tree=0, token=12:
['Orlando', 'PROPN', 13, 'nmod', 11]
NE parent token:
['WDW', 'PROPN', 14, 'compound', 13]
parent node subtree

...

txt=Retweeted ABC News ( @ABC ): Streets in San Juan, Puerto Rico remain flooded days after #Maria made landfall as a... <URL>
candidate 0=San Juan
anchor NE candidates = Puerto Rico
data NE tree=[['San', 'PROPN', 9, 'compound', 8], ['Juan', 'PROPN', 11, 'compound', 9]]
NE parse token at tree=0, token=10:
['Juan', 'PROPN', 11, 'compound', 9]
NE parent token:
['Rico', 'PROPN', 7, 'pobj', 11]
parent node subtree [['San', 'PROPN', 9, 'compound', 8], ['Juan', 'PROPN', 11, 'compound', 9], ['Puerto', 'PROPN', 11, 'compound', 10]]
txt=Woohoo, Ponce and adjacent areas, help is coming! This is an update from mi tio in Tampa, who is monitoring faith organizations. #PuertoRico <URL>
candidate 0=Ponce
anchor NE candidates = Tampa
data NE tree=[['Ponce', 'PROPN', 5, 'nsubj', 1]]
NE parse token at tree=0, token=2:
['Ponce', 'PROPN', 5, 'nsubj', 1]
NE parent token:
['help', 'VERB', 7, 'nsubj', 5]
NE=Ponce subtree=[['Woohoo', 'PROPN', 1, 'compound', 0], ['and', 'CCONJ', 1, 'cc', 2], ['adjacent', 'ADJ'

txt=More than 80% of our customers in Bay, Franklin, Gulf, Jefferson & Wakulla counties lost power as #Michael roared on shore as a cat. 4 hurricane. Damage assessment & repairs to the electric system are underway in areas that crews are able to access. <URL>
candidate 0=Bay
anchor NE candidates = Franklin,Gulf,Jefferson
data NE tree=[['Bay', 'PROPN', 11, 'compound', 8]]
NE parse token at tree=0, token=9:
['Bay', 'PROPN', 11, 'compound', 8]
NE parent token:
['Jefferson', 'PROPN', 14, 'nmod', 11]
parent node subtree [['Bay', 'PROPN', 11, 'compound', 8], ['Franklin', 'PROPN', 11, 'compound', 9], ['Gulf', 'PROPN', 11, 'nmod', 10], ['&', 'CCONJ', 11, 'cc', 12], ['Wakulla', 'PROPN', 11, 'conj', 13]]
candidate 1=Franklin
anchor NE candidates = 
candidate 2=Gulf
anchor NE candidates = Franklin,Jefferson
data NE tree=[['Gulf', 'PROPN', 11, 'nmod', 10]]
NE parse token at tree=0, token=11:
['Gulf', 'PROPN', 11, 'nmod', 10]
NE parent token:
['Jefferson', 'PROPN', 14, 'nmod', 11]
parent node subtr

txt=DEVASTATING DAMAGE: This is my 2nd year living in Florida during Hurricane Season and the images don't get easier to watch. This picture shows the devastation left behind by #HurricaneMichael in Mexico Beach. Praying for the victims and their families. #PrayersforthePanhandle <URL>
candidate 0=Mexico Beach
anchor NE candidates = 
txt=Thank you, this Panama City Beach #hurricanemichael survivor appreciates what you are doing for us!! Your store in Santa Rosa Beach helped us today when we drove over from Panama City to purchase food/water/supplies for our friends and coworkers. The employees were so kind! <URL>
candidate 0=Panama City Beach
anchor NE candidates = Panama City
data NE tree=[['Panama', 'PROPN', 5, 'compound', 3], ['City', 'PROPN', 5, 'compound', 4], ['Beach', 'PROPN', 7, 'nmod', 5]]
NE parse token at tree=0, token=6:
['Beach', 'PROPN', 7, 'nmod', 5]
NE parent token:
['survivor', 'NOUN', 8, 'nsubj', 7]
parent node subtree [['this', 'DET', 7, 'det', 2], ['Panama', 'PROPN'

txt=#HappyMonday - If you think your Monday's bad... be reminded by the pics from #HurricaneMichael that it can ALWAYS get worse. Listen, we are here in Panama City, FL at groundZero delivering supplies to those impacted... <URL>
candidate 0=Panama City
anchor NE candidates = FL
data NE tree=[['Panama', 'PROPN', 7, 'compound', 5], ['City', 'PROPN', 7, 'compound', 6]]
NE parse token at tree=1, token=7:
['City', 'PROPN', 7, 'compound', 6]
NE parent token:
['FL', 'PROPN', 4, 'pobj', 7]
parent node subtree [['Panama', 'PROPN', 7, 'compound', 5], ['City', 'PROPN', 7, 'compound', 6]]
txt=The people ravaged by #HurricaneMichael in Panama City, Florida and Mexico Beach, Florida need our help. Let’s do this by donating to the American Red Cross.
candidate 0=Panama City
anchor NE candidates = Florida,Florida
data NE tree=[['Panama', 'PROPN', 7, 'compound', 6], ['City', 'PROPN', 8, 'compound', 7]]
NE parse token at tree=0, token=8:
['City', 'PROPN', 8, 'compound', 7]
NE parent token:
['Florida', 

txt=One home in Mexico Beach, Florida, appeared largely untouched amid the incredible destruction of #HurricaneMichael . And its owners, Lebron Lackey and his uncle, Russell King, say it's no… <URL>
candidate 0=Mexico Beach
anchor NE candidates = Florida
data NE tree=[['Mexico', 'PROPN', 4, 'compound', 3], ['Beach', 'PROPN', 5, 'compound', 4]]
NE parse token at tree=0, token=5:
['Beach', 'PROPN', 5, 'compound', 4]
NE parent token:
['Florida', 'PROPN', 2, 'pobj', 5]
parent node subtree [['Mexico', 'PROPN', 4, 'compound', 3], ['Beach', 'PROPN', 5, 'compound', 4]]
txt=Hey guys... PLEASE come to Panama City, Fl. We need some laughs after #HurricaneMichael The beach didn't get hit bad at all. There are many venues. Club La Vila is a huge one. We'd love to see ya'll. Please consider it.
candidate 0=Panama City
anchor NE candidates = Fl
data NE tree=[['Panama', 'PROPN', 5, 'compound', 3], ['City', 'PROPN', 5, 'compound', 4]]
NE parse token at tree=1, token=5:
['City', 'PROPN', 5, 'compound', 

Great! We see that the false negatives are unavailable from the parses, so we did the best that we could. We may have to revisit the conjunction + state rule.

What is the precision/recall for each of the subclause classes:

1. subclause_state
2. subclause_descriptor

In [394]:
annotated_anchor_data = pd.read_csv('../../data/mined_tweets/combined_tweet_tag_data_NE_flat_anchor_examples.tsv', sep='\t', index_col=False)
annotated_anchor_NEs = annotated_anchor_data[annotated_anchor_data.loc[:, 'context_subclause']==1].loc[:, 'NE']
annotated_anchor_txt = annotated_anchor_data[annotated_anchor_data.loc[:, 'context_subclause']==1].loc[:, 'txt']
display(annotated_anchor_data.head())
context_subclause_types = [x for x in annotated_anchor_data.columns if 'context_subclause_' in x]
print(annotated_anchor_data.loc[:, context_subclause_types].sum(axis=0))

,NE,txt,data_name,context_subclause,context_subclause_state,context_subclause_descriptor,context_subclause_compound,context_subclause_list
0,new_bern,RT @EdValleeWx: Our models specifically used f...,florence,0.0,1.0,0.0,0.0,0.0
1,wilmington,RT @WMO: Hurricane #Florence is likely to make...,florence,1.0,1.0,0.0,0.0,0.0
2,cape_fear,RT @ABC: LATEST: Hurricane #Florence a Categor...,florence,0.0,1.0,0.0,0.0,0.0
3,fayetteville,You put your lips on them cigars more than you...,florence,0.0,1.0,0.0,0.0,0.0
4,wilmington,RT @FoxNewsResearch: #HurricaneFlorence - late...,florence,0.0,1.0,0.0,0.0,0.0


context_subclause_state         96.0
context_subclause_descriptor    12.0
context_subclause_compound       3.0
context_subclause_list           4.0
dtype: float64


Let's rewrite the descriptor detection code to output the different types of context.

In [556]:
def detect_subclause_anchor_by_type(data, valid_var='valid_loc', 
                                    anchor_var='max_population', NE_var='NE', 
                                    data_name_var='data_name_fixed',
                                    child_dep_types=['acl', 'appos', 'prep', 'nummod'],
                                    parent_dep_types=['nmod', 'compound', 'appos'],
                                    parent_state_dep='nmod',
                                    parent_compound_dep='compound',
                                    verbose=False):
    data_NEs = data.loc[:, NE_var].values
    # fix format to match parse
    data_NEs = [x.replace('_', ' ') for x in data_NEs]
    data_valid = data.loc[:, valid_var].values
    data_NEs_valid = [x for x,y in zip(data_NEs, data_valid) if y==1]
    data_anchor_vals = data.loc[:, anchor_var].values
    data_txt = data.loc[:, 'txt'].iloc[0]
    data_txt_clean = data_txt.lower()
#     subclause_state = []
    subclause_descriptor = []
    subclause_state = []
#     subclause_compound = []
#     subclause_list = []
    trees = data.loc[:, 'parse'].iloc[0]
    if(verbose):
        print('full parse %s'%(str(trees)))
    tree_graphs = [extract_full_tree(t) for t in trees]
    tree_ctr = 0
    tree_token_ctr = 0
    data_name = data.loc[:, data_name_var].iloc[0]
    states = DATA_NAME_STATES_LOOKUP[data_name]
    state_matcher = DATA_NAME_STATES_MATCHERS[data_name]
    for i, data_NE in enumerate(data_NEs_valid):
        if(verbose):
            print('candidate %d=%s'%(i, data_NE))
        anchor_val_i = data_anchor_vals[i]
        anchor_NE_candidates = [x for x,y in zip(data_NEs, data_anchor_vals) if y > anchor_val_i]
        if(verbose):
            print('anchor NE candidates = %s'%(','.join(anchor_NE_candidates)))
        subclause_state_i = 0
        subclause_descriptor_i = 0
#         subclause_compound_i = 0
#         subclause_list_i = 0
        ## easiest case: LOC, STATE pattern
        data_NE_state_matcher = re.compile('|'.join(['%s\s?,?\s?%s'%(data_NE, x) for x in states]).lower())
        subclause_state_i = int(data_NE_state_matcher.search(data_txt_clean) is not None)
        if(len(anchor_NE_candidates) > 0):
            # add whitespace to candidates to avoid matching substrings e.g. "FL" doesn't match "FLOW"
            anchor_NE_candidate_matcher = re.compile('|'.join([' %s |^%s | %s$|^%s$'%((x,)*4) for x in anchor_NE_candidates]))
            data_NE_tree, data_NE_subtree, tree_ctr, tree_token_ctr = extract_NE_subtree(data_NE, trees, tree_graphs, tree_ctr, tree_token_ctr)
            ## parent test: test if sibling contains state
            if(len(data_NE_tree) > 0):
                if(verbose):
                    print('data NE tree=%s'%(str(data_NE_tree)))
                # find parent node for noun, preposition tests
                # parent node => index that is not included in NE indices
                data_NE_tree_idx = [x[4] for x in data_NE_tree]
                NE_child_nodes = [x for x in data_NE_tree if x[2] not in data_NE_tree_idx]
                if(len(NE_child_nodes) > 0):
                    highest_child_node = NE_child_nodes[0]
                    parent_node_idx = highest_child_node[2]
                    parent_node_dep = highest_child_node[3]
                    parent_node = trees[tree_ctr][parent_node_idx]
                    if(verbose):
                        print('NE parse token at tree=%d, token=%d:'%(tree_ctr, tree_token_ctr))
                        print(str(trees[tree_ctr][tree_token_ctr-1]))
                        print('NE parent token:')
                        print(str(parent_node))
                    # if parent is connected by valid dep, find children and look for state mention
                    if(parent_node_dep in parent_dep_types):
                        parent_node_subtree_idx = get_subtree(parent_node[4], tree_graphs[tree_ctr])
                        # include parent in subtree
                        parent_node_subtree = [x for x in trees[tree_ctr] if x[4] in parent_node_subtree_idx or x[4]==parent_node_idx]
                        parent_node_subtree_str = ' '.join([x[0] for x in parent_node_subtree]).lower()
                        if(verbose):
                            print('parent node subtree %s'%(str(parent_node_subtree)))
                            print('parent node subtree str %s'%(parent_node_subtree_str))
#                         if(verbose):
#                             print('matching states %s on tree %s'%(state_matcher.pattern, parent_node_subtree_str.lower()))
                        ## collapse compound/nmod because they're arbitrary
#                         if(parent_node_dep in parent_dep_types):
#                         subclause_state_i = int(state_matcher.search(parent_node_subtree_str) is not None) 
#                         elif(parent_node_dep == parent_compound_dep):
#                         subclause_compound_i = int(state_matcher.search(parent_node_subtree_str) is not None) 
#                         parent_subclause_anchor_i = int(state_matcher.search(parent_node_subtree_str) is not None)
            
            ## child test: test if child contains anchor
            if(len(data_NE_subtree) > 0):
                if(verbose):
                    print('NE=%s subtree=%s'%(data_NE, str(data_NE_subtree)))
                ## filter for allowed dependency trees
                ## find dep type for highest node (lowest parent index) in subtree
                min_node_dep_idx = min(data_NE_subtree, key=lambda x: x[2])
                min_node_deps = [x[3] for x in data_NE_subtree if x[2]==min_node_dep_idx[2]]
                if(verbose):
                    print('min node deps %s'%(str(min_node_deps)))
                subtree_str = ' '.join([x[0] for x in data_NE_subtree])
                if(len(set(min_node_deps) & set(child_dep_types)) > 0):
                    ## look for NE in phrase
                    if(verbose):
                        print('subtree = %s'%(subtree_str))
                    subclause_descriptor_i = int(anchor_NE_candidate_matcher.search(subtree_str) is not None)
                    # sometimes the state comes as an appositive
#                     subclause_state_i = int(state_matcher.search(subtree_str.lower()) is not None)
                    ## conjuction is a special case: anchor NE must occur 
                    ## in format NE CONJ NE, state
#                 elif('conj' in min_node_deps):
#                     subclause_list_i = int(state_matcher.search(subtree_str.lower()) is not None)
        subclause_state.append(subclause_state_i)
        subclause_descriptor.append(subclause_descriptor_i)
#         subclause_compound.append(subclause_compound_i)
#         subclause_list.append(subclause_list_i)
    subclause_state = pd.Series(subclause_state, index=data_NEs_valid)
    subclause_descriptor = pd.Series(subclause_descriptor, index=data_NEs_valid)
#     subclause_compound = pd.Series(subclause_compound, index=data_NEs_valid)
#     subclause_list = pd.Series(subclause_list, index=data_NEs_valid)
#     return subclause_state, subclause_descriptor, subclause_compound, subclause_list
    return subclause_state, subclause_descriptor

Let's add the predicted subclause types to the annotated data, get a second set of annotations, and then compute precision/recall over the majority labels.

In [554]:
annotated_data_i

NE                                                                        mayfair
txt                             Homes in the Mayfair neighborhood of Lumberton...
data_name                                                                florence
context_subclause                                                               1
context_subclause_state                                                         0
context_subclause_descriptor                                                    0
context_subclause_compound                                                      1
context_subclause_list                                                          0
context_subclause_any                                                           1
Name: 0, dtype: object

In [562]:
## load annotated
annotated_anchor_data = pd.read_csv('../../data/mined_tweets/combined_tweet_tag_data_NE_flat_anchor_examples.tsv', sep='\t', index_col=False)
subclause_types = ['state', 'descriptor']
annotated_anchor_data_preds = []
for idx_i, annotated_data_i in annotated_anchor_data.iterrows():
    txt_i = annotated_data_i.loc['txt']
    data_i = anchor_data[anchor_data.loc[:, 'txt']==txt_i]
    NE_i = annotated_data_i.loc['NE']
    data_name_i = data_i.loc[:, 'data_name_fixed'].iloc[0]
    state_gold_i = annotated_data_i.loc['context_subclause_state']
    descriptor_gold_i = annotated_data_i.loc['context_subclause_descriptor']
    # restrict to data where NE only occurs once per status
    if(len([x for x in data_i.loc[:, 'NE'] if x.replace(' ', '_').lower()==NE_i]) == 1):
#         print('testing NE %s, txt:%s'%(NE_i, txt_i))
        subclause_state_i, subclause_descriptor_i = detect_subclause_anchor_by_type(data_i, verbose=False)
        subclause_state_i.index = [x.replace(' ', '_').lower() for x in subclause_state_i.index]
        subclause_descriptor_i.index = [x.replace(' ', '_').lower() for x in subclause_descriptor_i.index]
        subclause_state_i = subclause_state_i.loc[NE_i]
        subclause_descriptor_i = subclause_descriptor_i.loc[NE_i]
        annotated_anchor_data_preds.append([data_i.loc[:, 'id'].iloc[0], data_name_i, NE_i, txt_i, state_gold_i, descriptor_gold_i, subclause_state_i, subclause_descriptor_i])
annotated_anchor_data_preds = pd.DataFrame(annotated_anchor_data_preds, columns=['id', 'data_name', 'NE', 'txt', 'state_gold', 'descriptor_gold', 'state_pred', 'descriptor_pred'])
annotated_anchor_data_pred_out_file = '../../data/mined_tweets/NE_anchor_examples_with_pred.tsv'
annotated_anchor_data_preds.to_csv(annotated_anchor_data_pred_out_file, sep='\t', index=False)

In [480]:
## load annotated
annotated_anchor_data = pd.read_csv('../../data/mined_tweets/combined_tweet_tag_data_NE_flat_anchor_examples.tsv', sep='\t', index_col=False)
annotated_anchor_data_any_context = annotated_anchor_data[annotated_anchor_data.loc[:, 'context_subclause_any']==1]
## track all types of anchors
subclause_state = []
subclause_descriptor = []
# subclause_compound = []
# subclause_list = []
subclause_state_gold = []
subclause_descriptor_gold = []
# subclause_compound_gold = []
# subclause_list_gold = []
subclause_gold_any = []
# subclause_preds = [subclause_state, subclause_descriptor, subclause_compound, subclause_list]
# subclause_gold = [subclause_state_gold, subclause_descriptor_gold, subclause_compound_gold, subclause_list_gold]
subclause_preds = [subclause_state, subclause_descriptor]
subclause_gold = [subclause_state_gold, subclause_descriptor_gold]
subclause_types = ['state', 'descriptor']
for idx_i, annotated_data_i in annotated_anchor_data.iterrows():
    txt_i = annotated_data_i.loc['txt']
    data_i = anchor_data[anchor_data.loc[:, 'txt']==txt_i]
    NE_i = annotated_data_i.loc['NE']
    # restrict to data where NE only occurs once per status
    if(len([x for x in data_i.loc[:, 'NE'] if x.replace(' ', '_').lower()==NE_i]) == 1):
        print('testing NE %s, txt:%s'%(NE_i, txt_i))
        subclause_i = detect_subclause_anchor_by_type(data_i, verbose=True)
        subclause_gold_any_i = 0
        for subclause_j, subclause_type_j, subclause_pred_j, subclause_gold_j in zip(subclause_i, subclause_types, subclause_preds, subclause_gold):
            subclause_j.index = [x.replace(' ', '_').lower() for x in subclause_j.index]
            subclause_pred_j.append(subclause_j.loc[NE_i])
            subclause_gold_j.append(annotated_data_i.loc['context_subclause_%s'%(subclause_type_j)])
            subclause_gold_any_i = max(subclause_gold_any_i, annotated_data_i.loc['context_subclause_%s'%(subclause_type_j)])
            ## debugging: false negatives hurt recall
#             if(annotated_data_i.loc['context_subclause_%s'%(subclause_type_j)] - subclause_j.loc[NE_i] == 1.):
#                 print('false negative: NE=%s, type=%s'%(NE_i, subclause_type_j))
            ## debugging: false positives hurt precision
            if(subclause_j.loc[NE_i] - annotated_data_i.loc['context_subclause_%s'%(subclause_type_j)] == 1.):
                print('false positive: NE=%s, type=%s'%(NE_i, subclause_type_j))
        subclause_gold_any.append(subclause_gold_any_i)

## compute precision/recall
## use context_any for gold label in precision! because we don't care if there's overlap between categories
subclause_gold_any = np.array(subclause_gold_any)
for subclause_type, subclause_pred, subclause_gold in zip(subclause_types, subclause_preds, subclause_gold):
    print('testing subclause type=%s'%(subclause_type))
    subclause_gold = np.array(subclause_gold)
    subclause_pred = np.array(subclause_pred)
    # recall
    gold_idx = np.where(subclause_gold == 1.)
    gold_pred_diff = subclause_gold[gold_idx] - subclause_pred[gold_idx]
    recall = len(gold_pred_diff[gold_pred_diff == 0.]) / len(gold_pred_diff[gold_pred_diff >= 0.])
    print('%d/%d contexts detected'%(sum(subclause_pred[gold_idx]), sum(subclause_gold[gold_idx])))
    print('recall=%.3f'%(recall))
    # precision
    pred_idx = np.where(subclause_pred == 1.)
    all_pred_diff = subclause_pred[pred_idx] - subclause_gold_any[pred_idx]
    prec = len(all_pred_diff[all_pred_diff == 0.]) / len(all_pred_diff[all_pred_diff >= 0.])
    print('prec=%.3f'%(prec))

testing NE mayfair, txt:Homes in the Mayfair neighborhood of Lumberton, NC are underwater again, after being flooded in 2016 during Hurricane Matthew..and water levels continue to rise. #HurricaneFlorence #lumbertonnc #northcarolina #lumberriver @TheNOWtv <URL>
full parse [[['Homes', 'NOUN', 8, 'nsubj', 0], ['in', 'ADP', 0, 'prep', 1], ['the', 'DET', 4, 'det', 2], ['Mayfair', 'PROPN', 4, 'compound', 3], ['neighborhood', 'NOUN', 1, 'pobj', 4], ['of', 'ADP', 4, 'prep', 5], ['Lumberton', 'PROPN', 7, 'compound', 6], ['NC', 'PROPN', 5, 'pobj', 7], ['are', 'VERB', 8, 'ROOT', 8], ['underwater', 'ADJ', 8, 'acomp', 9], ['again', 'ADV', 8, 'advmod', 10], ['after', 'ADP', 8, 'prep', 11], ['being', 'VERB', 13, 'auxpass', 12], ['flooded', 'VERB', 11, 'pcomp', 13], ['in', 'ADP', 13, 'prep', 14], ['2016', 'NUM', 14, 'pobj', 15], ['during', 'ADP', 13, 'prep', 16], ['Hurricane', 'PROPN', 18, 'compound', 17], ['Matthew', 'PROPN', 16, 'pobj', 18], ['..', 'PUNCT', 8, 'punct', 19], ['and', 'CCONJ', 8, 'cc'

false positive: NE=kensington, type=descriptor
testing NE wilmington, txt:#hurricaneflorence in downtown Wilmington, NC @TheNOWtv @ Downtown Wilmington By Cape Fear River <URL>
full parse [[['#hurricaneflorence', 'PROPN', 0, 'ROOT', 0], ['in', 'ADP', 0, 'prep', 1], ['downtown', 'NOUN', 4, 'compound', 2], ['Wilmington', 'PROPN', 4, 'compound', 3], ['NC', 'PROPN', 5, 'compound', 4], ['@TheNOWtv', 'PROPN', 1, 'pobj', 5], ['@', 'ADP', 0, 'punct', 6], ['Downtown', 'PROPN', 8, 'compound', 7], ['Wilmington', 'PROPN', 8, 'ROOT', 8], ['By', 'ADP', 8, 'prep', 9], ['Cape', 'PROPN', 12, 'compound', 10], ['Fear', 'PROPN', 12, 'compound', 11], ['River', 'PROPN', 9, 'pobj', 12]]]
candidate 0=Wilmington
anchor NE candidates = NC
data NE tree=[['Wilmington', 'PROPN', 4, 'compound', 3]]
NE parse token at tree=0, token=4:
['Wilmington', 'PROPN', 4, 'compound', 3]
NE parent token:
['NC', 'PROPN', 5, 'compound', 4]
parent node subtree [['downtown', 'NOUN', 4, 'compound', 2], ['Wilmington', 'PROPN', 4, 'com

testing NE kinston, txt:Deputies from the Rockingham County Sheriff's Office are here in Kinston, NC helping out with issues caused by #flooding. #HurricaneFlorence #KinstonNC <URL>
full parse [[['Deputies', 'NOUN', 7, 'nsubj', 0], ['from', 'ADP', 0, 'prep', 1], ['the', 'DET', 6, 'det', 2], ['Rockingham', 'PROPN', 4, 'compound', 3], ['County', 'PROPN', 6, 'compound', 4], ["Sheriff's", 'PROPN', 6, 'compound', 5], ['Office', 'PROPN', 1, 'pobj', 6], ['are', 'VERB', 7, 'ROOT', 7], ['here', 'ADV', 7, 'advmod', 8], ['in', 'ADP', 7, 'prep', 9], ['Kinston', 'PROPN', 11, 'compound', 10], ['NC', 'PROPN', 9, 'pobj', 11], ['helping', 'VERB', 7, 'advcl', 12], ['out', 'PART', 12, 'prt', 13], ['with', 'ADP', 12, 'prep', 14], ['issues', 'NOUN', 14, 'pobj', 15], ['caused', 'VERB', 15, 'acl', 16], ['by', 'ADP', 16, 'agent', 17], ['#flooding', 'VERB', 17, 'pobj', 18], ['.', 'PUNCT', 7, 'punct', 19]], [['#HurricaneFlorence', 'PROPN', 1, 'compound', 0], ['#KinstonNC', 'PROPN', 1, 'ROOT', 1]]]
candidate 0=K

testing NE lumberton, txt:Houses sit in floodwater caused by #HurricaneFlorence , in this aerial picture, on the outskirts of Lumberton, North Carolina, via @Reuters photographer Jason Miczek See Reuters top photos from the last 24 hours: <URL>
full parse [[['Houses', 'NOUN', 1, 'nsubj', 0], ['sit', 'VERB', 1, 'ROOT', 1], ['in', 'ADP', 1, 'prep', 2], ['floodwater', 'NOUN', 2, 'pobj', 3], ['caused', 'VERB', 3, 'acl', 4], ['by', 'ADP', 4, 'agent', 5], ['#HurricaneFlorence', 'NOUN', 5, 'pobj', 6], ['in', 'ADP', 4, 'prep', 7], ['this', 'DET', 10, 'det', 8], ['aerial', 'ADJ', 10, 'amod', 9], ['picture', 'NOUN', 7, 'pobj', 10], ['on', 'ADP', 10, 'prep', 11], ['the', 'DET', 13, 'det', 12], ['outskirts', 'NOUN', 11, 'pobj', 13], ['of', 'ADP', 13, 'prep', 14], ['Lumberton', 'PROPN', 17, 'compound', 15], ['North', 'PROPN', 17, 'compound', 16], ['Carolina', 'PROPN', 14, 'pobj', 17], ['via', 'ADP', 1, 'prep', 18], ['@Reuters', 'PROPN', 20, 'compound', 19], ['photographer', 'NOUN', 18, 'pobj', 20],

testing NE lumberton, txt:Thanks to the amazing generosity of people in and around the Triangle, @WRAL helped fill 5 buses, 5 trailers, 3 vans and a small truck with supplies for #Florence victims. 130,500 lbs in supplies will be delivered next week to Wilmington, Lumberton and New Bern. #wral
full parse [[['Thanks', 'NOUN', 13, 'npadvmod', 0], ['to', 'ADP', 0, 'prep', 1], ['the', 'DET', 4, 'det', 2], ['amazing', 'ADJ', 4, 'amod', 3], ['generosity', 'NOUN', 1, 'pobj', 4], ['of', 'ADP', 4, 'prep', 5], ['people', 'NOUN', 5, 'pobj', 6], ['in', 'ADP', 4, 'prep', 7], ['and', 'CCONJ', 7, 'cc', 8], ['around', 'ADP', 7, 'conj', 9], ['the', 'DET', 12, 'det', 10], ['Triangle', 'PROPN', 12, 'compound', 11], ['@WRAL', 'PROPN', 9, 'pobj', 12], ['helped', 'VERB', 13, 'ROOT', 13], ['fill', 'VERB', 13, 'xcomp', 14], ['5', 'NUM', 16, 'nummod', 15], ['buses', 'NOUN', 14, 'dative', 16], ['5', 'NUM', 18, 'nummod', 17], ['trailers', 'NOUN', 14, 'dobj', 18], ['3', 'NUM', 20, 'nummod', 19], ['vans', 'NOUN', 

...

testing NE tallahassee, txt:HurriCation Self Portrait 3. Nice work on the backdrop katebackdrops #hurricanemichaelmademedoit #longexposure @ Tallahassee, Florida <URL>
full parse [[['HurriCation', 'PROPN', 2, 'compound', 0], ['Self', 'PROPN', 2, 'compound', 1], ['Portrait', 'PROPN', 2, 'ROOT', 2], ['3', 'NUM', 2, 'nummod', 3], ['.', 'PUNCT', 2, 'punct', 4]], [['Nice', 'ADJ', 1, 'amod', 0], ['work', 'NOUN', 1, 'ROOT', 1], ['on', 'ADP', 1, 'prep', 2], ['the', 'DET', 4, 'det', 3], ['backdrop', 'NOUN', 5, 'compound', 4], ['katebackdrops', 'NOUN', 2, 'pobj', 5], ['#hurricanemichaelmademedoit', 'X', 7, 'compound', 6], ['#longexposure', 'PROPN', 7, 'ROOT', 7], ['@', 'ADP', 7, 'punct', 8], ['Tallahassee', 'PROPN', 10, 'compound', 9], ['Florida', 'PROPN', 10, 'ROOT', 10]]]
candidate 0=Tallahassee
anchor NE candidates = Florida
data NE tree=[['Tallahassee', 'PROPN', 10, 'compound', 9]]
NE parse token at tree=1, token=10:
['Tallahassee', 'PROPN', 10, 'compound', 9]
NE parent token:
['Florida', 'P

testing NE panama_city_beach, txt:All day we've all seen the horrific landscape in Panama City Beach and Mexico Beach, Florida and other parts of the Panhandle. But every new drone or aerial shot continues to stun. #Michael was swift. In the morning things were there. In the afternoon, they weren't. <URL>
full parse [[['All', 'DET', 1, 'det', 0], ['day', 'NOUN', 4, 'npadvmod', 1], ["we've", 'VERB', 3, 'nmod', 2], ['all', 'DET', 4, 'nsubj', 3], ['seen', 'VERB', 4, 'ROOT', 4], ['the', 'DET', 7, 'det', 5], ['horrific', 'ADJ', 7, 'amod', 6], ['landscape', 'NOUN', 4, 'dobj', 7], ['in', 'ADP', 7, 'prep', 8], ['Panama', 'PROPN', 11, 'compound', 9], ['City', 'PROPN', 11, 'nmod', 10], ['Beach', 'PROPN', 8, 'pobj', 11], ['and', 'CCONJ', 11, 'cc', 12], ['Mexico', 'PROPN', 14, 'compound', 13], ['Beach', 'PROPN', 11, 'conj', 14], ['Florida', 'PROPN', 11, 'conj', 15], ['and', 'CCONJ', 11, 'cc', 16], ['other', 'ADJ', 18, 'amod', 17], ['parts', 'NOUN', 11, 'conj', 18], ['of', 'ADP', 18, 'prep', 19], [

testing NE panama_city_beach, txt:Heard you guys are bringing prepaid phones and charging stations. Please don’t do this on Panama City Beach. They have electricity and access to Destin. Come into Panama City, Callaway, Lynn Haven @TMobile @TMobileHelp #hurricanemichael
full parse [[['Heard', 'ADP', 4, 'mark', 0], ['you', 'PRON', 2, 'nmod', 1], ['guys', 'NOUN', 4, 'nsubj', 2], ['are', 'VERB', 4, 'aux', 3], ['bringing', 'VERB', 4, 'ROOT', 4], ['prepaid', 'ADJ', 6, 'amod', 5], ['phones', 'NOUN', 4, 'dobj', 6], ['and', 'CCONJ', 4, 'cc', 7], ['charging', 'VERB', 4, 'conj', 8], ['stations', 'NOUN', 8, 'dobj', 9], ['.', 'PUNCT', 4, 'punct', 10]], [['Please', 'INTJ', 1, 'intj', 0], ['don', 'VERB', 1, 'ROOT', 1], ['’', 'NOUN', 3, 'compound', 2], ['t', 'NOUN', 1, 'dobj', 3], ['do', 'VERB', 1, 'conj', 4], ['this', 'DET', 4, 'dobj', 5], ['on', 'ADP', 4, 'prep', 6], ['Panama', 'PROPN', 9, 'compound', 7], ['City', 'PROPN', 9, 'compound', 8], ['Beach', 'PROPN', 6, 'pobj', 9], ['.', 'PUNCT', 1, 'punc

false positive: NE=panama_city, type=compound
testing NE youngstown, txt:Day #2 of Hurricane #Michael Damage Surveys - Unbelievable tree damage. We surveyed spots in Southport, Youngstown, & Resota Beach (north of Lynn Haven & Panama City) where all trees were snapped, uprooted, or bent/twisted. Pictures do NOT do the damage justice at all... <URL>
full parse [[['Day', 'NOUN', 0, 'ROOT', 0], ['#', 'NOUN', 2, 'nmod', 1], ['2', 'NUM', 0, 'nummod', 2], ['of', 'ADP', 2, 'prep', 3], ['Hurricane', 'PROPN', 5, 'compound', 4], ['#Michael', 'PROPN', 10, 'compound', 5], ['Damage', 'NOUN', 7, 'nmod', 6], ['Surveys', 'NOUN', 10, 'compound', 7], ['Unbelievable', 'ADJ', 9, 'compound', 8], ['tree', 'NOUN', 10, 'compound', 9], ['damage', 'NOUN', 3, 'pobj', 10], ['.', 'PUNCT', 0, 'punct', 11]], [['We', 'PRON', 1, 'nsubj', 0], ['surveyed', 'VERB', 1, 'ROOT', 1], ['spots', 'NOUN', 1, 'dobj', 2], ['in', 'ADP', 2, 'prep', 3], ['Southport', 'PROPN', 5, 'compound', 4], ['Youngstown', 'PROPN', 3, 'pobj', 5], 

testing NE mexico_beach, txt:Want to see the real #AHSApocalypse ? Come down here to Mexico beach or Panama City, FL! #HurricaneMichael #hurricanemichael2018 @CNN @weatherchannel
full parse [[['Want', 'VERB', 0, 'ROOT', 0], ['to', 'PART', 2, 'aux', 1], ['see', 'VERB', 0, 'xcomp', 2], ['the', 'DET', 5, 'det', 3], ['real', 'ADJ', 5, 'amod', 4], ['#AHSApocalypse', 'NOUN', 2, 'dobj', 5], ['?', 'PUNCT', 0, 'punct', 6]], [['Come', 'VERB', 0, 'ROOT', 0], ['down', 'PART', 0, 'prt', 1], ['here', 'ADV', 0, 'advmod', 2], ['to', 'ADP', 0, 'prep', 3], ['Mexico', 'PROPN', 5, 'compound', 4], ['beach', 'NOUN', 3, 'pobj', 5], ['or', 'CCONJ', 5, 'cc', 6], ['Panama', 'PROPN', 9, 'compound', 7], ['City', 'PROPN', 9, 'compound', 8], ['FL', 'PROPN', 5, 'conj', 9], ['!', 'PUNCT', 0, 'punct', 10]], [['#HurricaneMichael', 'PROPN', 2, 'compound', 0], ['#hurricanemichael2018', 'PUNCT', 2, 'nsubj', 1], ['@CNN', 'PROPN', 2, 'ROOT', 2], ['@weatherchannel', 'X', 2, 'punct', 3]]]
candidate 0=Mexico beach
anchor NE ca

In [473]:
print(subclause_estimate[estimate_idx])
print(subclause_gold_any[estimate_idx])

[1 1 1 1]
[1 1 1 1]


### Test precision on full data

Let's label all the data with the high-precision anchors, get a random sample, and re-label to ensure that the precision doesn't take a big hit.

In [493]:
combined_data_parsed_flat.columns

Index(['id', 'txt', 'data_name_fixed', 'username', 'date', 'lang', 'NE',
       'NE_type', 'NE_LOC', 'valid_loc', 'NE_fixed', 'has_descriptor',
       'NE_fixed_clean', 'max_population', 'max_alternate_name_count',
       'max_population_anchor', 'max_population_diff',
       'max_alternate_name_count_anchor', 'max_alternate_name_count_diff',
       'parse'],
      dtype='object')

In [ ]:
subclause_anchor_data_combined = []
id_var = 'id'
id_var_unique = combined_data_parsed_flat.loc[:, id_var].nunique()
subclause_anchor_var = 'subclause_anchor'
subclause_types = ['state', 'descriptor', 'compound', 'list']
parse_data_cols = ['txt', 'parse', 'data_name_fixed', 'valid_loc', 'has_descriptor']
NE_var = 'NE_fixed'
for i, (id_i, data_i) in enumerate(combined_data_parsed_flat.groupby(id_var)):
    subclause_i = detect_subclause_anchor_by_type(data_i, verbose=False)
    if(len(subclause_i[0]) > 0):
        data_i = data_i[data_i.loc[:, 'valid_loc']==1]
    #     for subclause_j in subclause_i:
    #         subclause_j.index = [x.replace(' ', '_').lower() for x in subclause_j.index]
        anchor_data_i = pd.concat(subclause_i, axis=1)
        anchor_data_i.columns = subclause_types
        anchor_data_i = anchor_data_i.reset_index(inplace=False).rename(columns={'index' : NE_var})
        anchor_data_i = anchor_data_i.assign(**{subclause_anchor_var : anchor_data_i.loc[:, subclause_types].max(axis=1)})
#         print('anchor data N=%d, data N=%d'%(anchor_data_i.shape[0], data_i.shape[0]))
        anchor_data_i.index = data_i.index
        anchor_data_i = pd.concat([anchor_data_i, data_i.loc[:, parse_data_cols]], axis=1)
        subclause_anchor_data_combined.append(anchor_data_i)
    if(i % 1000 == 0):
        print('processed %d/%d ID groups'%(i, id_var_unique))

In [534]:
subclause_anchor_data_combined = pd.concat(subclause_anchor_data_combined, axis=0)

In [535]:
print('%d samples'%(subclause_anchor_data_combined.shape[0]))

158863 samples


In [536]:
print(subclause_anchor_data_combined.columns)
display(subclause_anchor_data_combined.loc[:, subclause_types + [subclause_anchor_var]].sum(axis=0))

Index(['NE_fixed', 'state', 'descriptor', 'compound', 'list',
       'subclause_anchor', 'txt', 'parse', 'data_name_fixed', 'valid_loc',
       'has_descriptor'],
      dtype='object')


state               16950
descriptor           1103
compound            13906
list                  532
subclause_anchor    19193
dtype: int64

Let's get a random sample of each type and annotate for precision.

In [542]:
sample_size = 10
group_var = 'data_name_fixed'
subclause_anchor_data_combined_samples = []
sample_usecols = ['NE_fixed', 'txt', group_var] + subclause_types
for i, data_i in subclause_anchor_data_combined.groupby(group_var):
    samples_i = [data_i[data_i.loc[:, x]==1].assign(**{x : 1}) for x in subclause_types]
    samples_i = [x.loc[np.random.choice(x.index, min(sample_size, x.shape[0]), replace=False)] for x in samples_i]
    subclause_anchor_data_combined_samples += samples_i
subclause_anchor_data_combined_samples = pd.concat(subclause_anchor_data_combined_samples, axis=0)
subclause_anchor_data_combined_samples = subclause_anchor_data_combined_samples.loc[:, sample_usecols]
## add gold cols for annotation
subclause_anchor_data_combined_samples = subclause_anchor_data_combined_samples.assign(**{'%s_gold'%(x) : 0 for x in subclause_types})
print(subclause_anchor_data_combined_samples.groupby(group_var).apply(lambda x: x.loc[:, subclause_types].sum(axis=0)))
display(subclause_anchor_data_combined_samples.head())
## write to file to annotate! so exciting!!
sample_out_file = '../../data/mined_tweets/combined_tweet_tag_data_NE_flat_anchor_examples_by_type.tsv'
subclause_anchor_data_combined_samples.to_csv(sample_out_file, sep='\t', index=False)

                 state  descriptor  compound  list
data_name_fixed                                   
florence            21          10        23    11
harvey              21          10        19    10
irma                20          11        16    10
maria               22          10        14    10
michael             22          10        18    10


,NE_fixed,txt,data_name_fixed,state,descriptor,compound,list,state_gold,descriptor_gold,compound_gold,list_gold
460913,Lumberton,Leaving Lumberton... Trailers found a low wate...,florence,1,0,0,0,0,0,0,0
454417,New Bern,"The scene at the waterfront park in New Bern, ...",florence,1,0,1,0,0,0,0,0
461879,Aurora,Update from Capt. Stephan Wildish who is servi...,florence,1,0,1,0,0,0,0,0
454282,Wilmington,RT @WFLALeigh: WOW! The potential rainfall tot...,florence,1,0,1,0,0,0,0,0
459450,Greensboro,My version of the #Poblanos Chicken Soup! #hom...,florence,1,0,1,0,0,0,0,0


After annotating...let's evaluate the precision.

In [545]:
subclause_anchor_data_combined_samples.head()

,NE_fixed,txt,data_name_fixed,state,descriptor,compound,list,state_gold,descriptor_gold,compound_gold,list_gold,any_gold
0,Lumberton,Leaving Lumberton... Trailers found a low wate...,florence,1,0,0,0,1,0,0,0,1
1,New Bern,"The scene at the waterfront park in New Bern, ...",florence,1,0,1,0,1,0,0,0,1
2,Aurora,Update from Capt. Stephan Wildish who is servi...,florence,1,0,1,0,1,0,0,0,1
3,Wilmington,RT @WFLALeigh: WOW! The potential rainfall tot...,florence,1,0,1,0,1,0,0,0,1
4,Greensboro,My version of the #Poblanos Chicken Soup! #hom...,florence,1,0,1,0,1,0,0,0,1


In [550]:
def compute_precision_for_subclause_types(data, subclause_types, gold_col='any_gold'):
    prec_by_type = []
#     data_gold = data[data.loc[:, gold_col]==1.]
    for x in subclause_types:
        diff_x = data.loc[:, x] - data.loc[:, gold_col]
        prec_x = len(diff_x[diff_x == 0.]) / len(diff_x[diff_x >= 0.])
        prec_by_type.append(prec_x)
    prec_by_type = pd.Series(prec_by_type, index=subclause_types)
    return prec_by_type
subclause_anchor_data_combined_samples = pd.read_csv(sample_out_file, sep='\t', index_col=False)
display(subclause_anchor_data_combined_samples.head())
# compute any_gold
gold_col = 'any_gold'
if(gold_col not in subclause_anchor_data_combined_samples.columns):
    subclause_anchor_data_combined_samples = subclause_anchor_data_combined_samples.assign(**{'any_gold' : subclause_anchor_data_combined_samples.loc[:, ['%s_gold'%(x) for x in subclause_types]].max(axis=1)})
# per-group
group_var = 'data_name_fixed'
group_sample_prec = subclause_anchor_data_combined_samples.groupby(group_var).apply(lambda x: compute_precision_for_subclause_types(x, subclause_types))
display(group_sample_prec)
# overall
data_sample_prec = compute_precision_for_subclause_types(subclause_anchor_data_combined_samples, subclause_types)
display(data_sample_prec)
## we want: data_name | anchor_type | precision
## overall | anchor_type | precision

,NE_fixed,txt,data_name_fixed,state,descriptor,compound,list,state_gold,descriptor_gold,compound_gold,list_gold
0,Lumberton,Leaving Lumberton... Trailers found a low wate...,florence,1,0,0,0,1,0,0,0
1,New Bern,"The scene at the waterfront park in New Bern, ...",florence,1,0,1,0,1,0,0,0
2,Aurora,Update from Capt. Stephan Wildish who is servi...,florence,1,0,1,0,1,0,0,0
3,Wilmington,RT @WFLALeigh: WOW! The potential rainfall tot...,florence,1,0,1,0,1,0,0,0
4,Greensboro,My version of the #Poblanos Chicken Soup! #hom...,florence,1,0,1,0,1,0,0,0


,state,descriptor,compound,list
data_name_fixed,,,,
florence,0.965517,0.733333,0.896552,0.666667
harvey,0.947368,0.550000,0.944444,0.550000
irma,0.942857,0.666667,0.968750,0.650000
maria,1.000000,0.722222,1.000000,0.466667
michael,0.958333,0.916667,1.000000,0.916667


state         0.962733
descriptor    0.697674
compound      0.958621
list          0.634146
dtype: float64

### Old code

## Learn to classify information status
Let's get gold labels from LOC, DESCRIPTOR pairs and train a classifier using ELMO embeddings to predict descriptor, then apply that to the full data.

In [33]:
import sys
if('..' not in sys.path):
    sys.path.append('..')
from importlib import reload
import models.model_helpers
reload(models.model_helpers)
from models.model_helpers import load_elmo
elmo_model = load_elmo()
# dir(torch)

ImportError: cannot import name 'cuda'